## Coursera Capstone Project - The Battle of Neighborhoods
# Analysis of London residential housing market
#### January 2021 - Fabrizio Boffa

## Introduction

London is a fascinating city with many faces and, as a background of this project, we will assume that a group of stakeholders has already decided to invest in its real estate market.

Given that, the objective of this project is to find newly built residential properties that match their investment profile, on one hand, and possess the potential to increase their future value, on the other. We will accomplish this by defining a rating system based on the residential properties characteristics, on the relevant trends about the area they are located in and on their surrounding venues.

First, we will look for available datasets from which we can gather some insights about future housing market trends in various areas of London. Afterwards we will pick the most promising areas and look for residential unit on sale in their perimeter browsing housing specialized websites. We will then inspect each property surroundings using the Foursquare API to see what kind of venues are nearby and further characterize each property.

Combining all these information about local housing market trends, properties features and their surroundings, this document proposes and enforces a mathodology to approach the residential housing market so that an investor is finally able to make an aware buying decision. 

Even though this document is intended for a group of stakeholders that is interested in the London housig market for a real estate investment, it is also interesting for the small investor that wants to settle in London buying a new home with certain characteristics and surroundings and that has a good chance to increase its value in the future.

## Data

We will acquire data from different sources on three main subjects:

- Local Areas
- Actual residential properties on sales
- Surrounding venues

As local areas, we will use the London boroughs geographical subdivision and the main source of data will be the London Datastore https://data.london.gov.uk/. It has been created by the Greater London Authority (GLA) as a first step towards freeing London’s data. It will provide us with a huge amount of historical data and projections grouped by boroughs or wards on a wide range of topics.<br>
Looking at available London data at borough level, we will pick those that likely will have an impact on the housing market trend, namely:
- Housing market historical trends
- Demographic historical trend and predictions
- Income historical trend
- Future residential development
- Crimes historical trend
- Air pollution historical trends and predictions

As said, all the information have been acquired from London Datastore, apart from City of London Crimes data which comes from the UKCrimeStats website (www.ukcrimestats.com), an open data platform of the Economic Policy Centre (www.economicpolicycentre.com). In the methodology section of this document we will further describe every dataset analyzed that will be mainly in the time series form. Nevertheless, the common goal will be to produce a ratings system that synthesizes the impact of the described local feature on the future housing market value in that same area.


Afterwards, we will acquire data about actual residential properties analyzing one of the most importatnt UK website for properties for sales: www.rightmove.co.uk.<br>
This source will provide us with these information about actual residential property on sales in the London area:

- Type of residential unit
- Location coordinates
- Number of bedrooms
- Size
- Price

These data will give us the ability to estimate further property characteristics like "spaciousness" and "affordability" to better match investor preferences.

Finally, with the location coordinates acquired in the previous step, we will use the Foursquare API to acquire nearby venues. We will subdivide venues in these categories:

- Outdoors & Entertainment
- Services
- Food and Nightlife
- Transports

and count the number of venues nearby in each category to characterize the surroundings of each residential unit.


At this point, we will have all the information to completely rate each single property. We will then define a final rating system that we could match with an investor profile to obtain a personalized recomended buying list.

## Metodology

The metodolgy proposed is divided in two phases.

In the first phase we will assess the boroughs quality in relation to a potential residential investment.<br> 
We will analyze the various data collected to evaluate predictions for the imminent future. Based on these predictions and the impact of each feature on the housing market value, we will define for each borough a rating on each subject analyzed.<br>
Predictions about the imminent future are fundamental for our goal. We have to consider that the current value of each feature is correlated to the current housing market value, so by itself is not sufficient to give us the information we want, namely the likelihood of a property value increase. Indeed, the current value define both the current selling and buyng prices so it is impossible to gain information about what will be the prices in the future. In other words, if we invest today we want to find the borough where there's a high possibility of a value increase of our investment.

The mentioned rating will reflect the impact on the housing market of the selected study topics and for each one of them every borough will be graded from zero (worst) to one (best). A good rating will mean that feature will likely contribute to a positive increment of house market value in that borough.

The rating itself will be evaluated with the relative increment between the future value and the current value of the given feature with the formula: 

# <center>$\frac{v_f - v_c}{v_c}$</center>

We will assess the future values at one year from December 2020. <br>
With all the increments calculated for a given feature we will obtain the final ratings simply applying a min-max normalization in the zero-one range so that for every feature we will have a best borough rated at one and a worst rated at zero.

Almost all the dataset we will analyze in this phase are time series and some of them contain also predictive data. In the lack of a future value, we will extrapolate one using a polynomial regression model and, since we want to capture just the overall trend, we will keep a low polynomial degree.

All the ratings will be stored in a dataframe and we will perform clustering analysis using K-means and DBSCAN algorithm to obtain some clusters with interesting characteristics with the purpose of selecting some promising boroughs that we will use for the second phase.

In the second phase we will use a web scraping algorithm to acquire data about new residential property on sales in the most promising boroughs and run several iterations of the Foursquare "explore" API to find surrounding venues for each property.<br>
Based on the data gathered we will run several clustering algorithm to extract the most interesting groups of properties that are suitable for our goal.

In [ ]:
# LIBRARIES

import os
from dateutil.relativedelta import relativedelta
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
import json
import requests
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib import rcParams
import seaborn as sns
import folium
from shapely.geometry import shape, Point
from bokeh.io import show, output_notebook, curdoc
from bokeh.plotting import figure 
from bokeh.layouts import column as b_column
from bokeh.layouts import row as b_row
from bokeh.models import CustomJS, Legend, LegendItem, ColumnDataSource, Select, Label , Div, Title
from bokeh.themes import Theme
from geopy import distance
from IPython.core.display import HTML


In [ ]:
# GENERAL SETTINGS

# Pandas settings
pd.options.mode.chained_assignment = None

# Date settings
num_years_ml = 1 # How many years in the future we want to predict from machine learning models
num_years_pr = 1 # How many years in the future we want to pick data from (predictions already in datasets)
c_date = pd.Timestamp('2020-12') # Current month timestamp
c_year = pd.to_datetime(str(c_date.year)) # Current year timestamp

# Matplotlib settings
size_p = (8, 5) # plot size in inches
dpi_p = 100 # plt dpi
size_t = 16 # title size
size_an = 12  # Annotation and legend size
size_l = 2 # line width standard plots
size_s = 6 # size scatter plots points
font = 'times' # Also custom.css template must be updated
rcParams['font.family'] = 'serif'
rcParams['font.sans-serif'] = ['times']


# Colors settings
std_palette = 'RdYlGn'
bad_col = '#b30d26'
good_col = '#07753e'

# Bokeh settings
output_notebook(hide_banner = True)
curdoc().theme = Theme(json={'attrs': {

    # apply defaults to Figure properties
    'Figure': {
        'plot_width' : size_p[0] *dpi_p,
        'plot_height' : size_p[1] *dpi_p,
        'toolbar_location' : None
    },

    # apply defaults to Axis properties
    'Axis': {
        'axis_label_text_font_size' : '12pt',
        'axis_label_text_font' : 'times',
        'axis_label_text_font_style' : 'bold',
        'axis_label_text_color' : 'black',
        'major_label_text_font_size' : '12pt',
        'major_label_text_font' : font,
        'major_label_text_color' : 'black'
    },

    'Title': {
        'text_font_size' : '{}pt'.format(size_t),
        'text_font' : font,
        'text_font_style' : 'normal',
        'text_color' : 'black'
    },
     # apply defaults to Legend properties
    'Legend': {
        'label_text_font_size' : '9pt',
        'label_text_font' : font,
        'label_text_color' : 'black'
    },
    'Toolbar': {
        'active_drag' : None,
        'active_inspect' : None,
        'active_multi' : None,
        'active_scroll' : None,
        'active_tap' : None,
    }
}})


In [ ]:
# FUNCTIONS AND CLASSES

# Color palette function that defines color from y values
def colors_func(values):
    color_steps = 10
    normalized = (values - min(values)) / (max(values) - min(values)) # normalize the values to range [0, 1]
    indices = np.round(normalized * (color_steps - 1)).astype(np.int32) # convert to indices
    palette = sns.color_palette(std_palette, color_steps) # use the indices to get the colors
    return np.array(palette).take(indices, axis = 0)

# Class for time series analysis and plot
class time_series_class:
    def __init__(self, df, r_col1, r_col2 = None):
        self.df = df
        self.r_col1 = r_col1
        self.r_col2 = r_col2
    x_pl1 = []
    y_pl1 = []
    x_pl2 = []
    y_pl2 = []
    incr_st_list = []
    incr_lt_list = []
    
    # Calculate relative increases and define plotting data (for time series that already contain predictions)
    def rate(self, negative_feature = False):  
        self.x_pl1 = []
        self.y_pl1 = []
        self.incr_st_list = []
        self.x_pl1 = pd.Series(data = [c_year, c_year + relativedelta(years = num_years_pr)])
        self.y_pl1 = self.df[[c_year, c_year + relativedelta(years = num_years_pr)]].values
        
        for row in self.df.iterrows():
            self.incr_st_list.append((row[1][c_year + relativedelta(years = num_years_pr)] - row[1][c_year]) / row[1][c_year])
        df_ratings[self.r_col1] = MinMaxScaler().fit_transform(np.array(self.incr_st_list)[:, np.newaxis])
        if negative_feature == True:
            df_ratings[self.r_col1] = -df_ratings[self.r_col1] + 1
        df_ratings[self.r_col1] = round(df_ratings[self.r_col1], 2)
        
    # Calculate regressions, relative increases and define plotting data (for time series that do not contain predictions)
    # Can fit two different polynomial regressions and define relative plotting data for short term and lon term trend analysis
    def poly(self, pg_st, num_years_st, pg_lt = None, num_years_lt = None, negative_feature = False):
        self.x_pl1 = []
        self.y_pl1 = []
        self.x_pl2 = []
        self.y_pl2 = []
        self.incr_st_list = []
        self.incr_lt_list = []

        last_date = self.df.columns.max()
        try:
            first_date_st = c_date - relativedelta(years = num_years_st)
            df_st = self.df.drop(self.df.columns[0:self.df.columns.get_loc(first_date_st)], axis = 1)
        except:
            first_date_st = c_year - relativedelta(years = num_years_st)
            df_st = self.df.drop(self.df.columns[0:self.df.columns.get_loc(first_date_st)], axis = 1)
        x_fit_st = np.array(df_st.columns)[:, np.newaxis]
        x_pr_to_add_pts_list = [np.datetime64(c_date)]
        x_pr_to_add_pts_list = [np.datetime64(c_date)]
        for i in range(1,num_years_ml + 1):
            x_pr_to_add_pts_list.append(np.datetime64(c_date + relativedelta(years = i)))
        self.x_pl1 = np.append(x_fit_st,np.array(x_pr_to_add_pts_list))[:, np.newaxis]

        if (self.r_col2 != None):
            first_date_lt = c_date - relativedelta(years=num_years_lt)
            df_lt = self.df.drop(self.df.columns[0:self.df.columns.get_loc(first_date_lt)], axis = 1)
            x_fit_lt = np.array(df_lt.columns)[:, np.newaxis]
            self.x_pl2 = np.append(x_fit_lt,np.array(x_pr_to_add_pts_list))[:, np.newaxis]

        for i in range(1,num_years_ml+1):
            x_pr_to_add_pts_list.append(np.datetime64(c_date + relativedelta(years = i)))

        for borough in self.df.index:
            model_st = make_pipeline(MinMaxScaler(feature_range = (-1, 1)),PolynomialFeatures(pg_st), LinearRegression())
            model_st.fit(x_fit_st,df_st.loc[borough])
            y_pr_st = model_st.predict(self.x_pl1)
            self.y_pl1.append(y_pr_st)
            incr_st = (y_pr_st[len(y_pr_st) - 1] - y_pr_st[len(y_pr_st) - 1- num_years_ml]) / y_pr_st[len(y_pr_st) - 1 - num_years_ml]
            self.incr_st_list.append(incr_st)

            if (self.r_col2 != None):
                model_lt = make_pipeline(MinMaxScaler(feature_range = (-1, 1)),PolynomialFeatures(pg_lt), LinearRegression())
                model_lt.fit(x_fit_lt,df_lt.loc[borough])
                y_pr_lt = model_lt.predict(self.x_pl2)
                self.y_pl2.append(y_pr_lt)
                incr_lt = (y_pr_lt[len(y_pr_lt) - 1] - y_pr_lt[len(y_pr_lt) - 1 - num_years_ml])/ y_pr_lt[len(y_pr_lt) - 1 - num_years_ml]
                self.incr_lt_list.append(incr_lt)

        df_ratings[self.r_col1] = MinMaxScaler().fit_transform(np.array(self.incr_st_list)[:, np.newaxis])
        df_ratings[self.r_col1] = round(df_ratings[self.r_col1], 2)

        if (self.r_col2 != None):
            df_ratings[self.r_col2] = MinMaxScaler().fit_transform(np.array(self.incr_lt_list)[:, np.newaxis])
            df_ratings[self.r_col2] = round(df_ratings[self.r_col2], 2)
        
        if (negative_feature == True):
            df_ratings[self.r_col1] = -df_ratings[self.r_col1] + 1
            df_ratings[self.r_col1] = round(df_ratings[self.r_col1], 2)
            
        try:
            self.x_pl1 = self.x_pl1.ravel() #restore 1D for plotting 
            self.x_pl2 = self.x_pl2.ravel() #restore 1D for plotting
        except:
            pass
              
    def ts_plot(self, title_text, y_lbl, plot2):
        data_1 = self.df.stack().reset_index()
        data_1.columns = ['Borough', 'Date', 'Value']
        data_1c1 = data_1[data_1['Borough'] == df_ratings[self.r_col1].idxmax()]
        data_1c2 = data_1[data_1['Borough'] == df_ratings[self.r_col1].idxmin()]
        Overall_1 = ColumnDataSource(data = data_1)
        Curr_11 = ColumnDataSource(data = data_1c1)
        Curr_12 = ColumnDataSource(data = data_1c2)
        data_2 = pd.DataFrame(self.y_pl1, index = df_boroughs.index, columns = self.x_pl1).stack().reset_index()
        data_2.columns = [data_2.columns[0], 'Date', 'Value']
        data_2c1 = data_2[data_2['Borough'] == df_ratings[self.r_col1].idxmax()]
        data_2c2 = data_2[data_2['Borough'] == df_ratings[self.r_col1].idxmin()]
        Overall_2 = ColumnDataSource(data = data_2)
        Curr_21 = ColumnDataSource(data = data_2c1)
        Curr_22 = ColumnDataSource(data = data_2c2)
        plot_JS_code = """
        var borough = cb_obj.value
        sc.data['Date']=[]
        sc.data['Value']=[]
        for(var i = 0; i <= source.get_length(); i++){
            if (source.data['Borough'][i] == borough){
                sc.data['Date'].push(source.data['Date'][i])
                sc.data['Value'].push(source.data['Value'][i])
            }
        }   
        sc.change.emit();
        """
        callback_circle_1 = CustomJS(args = dict(source = Overall_1, sc = Curr_11), code = plot_JS_code)
        callback_line_11 = CustomJS(args = dict(source = Overall_1, sc = Curr_12), code = plot_JS_code)
        callback_circle_2 = CustomJS(args = dict(source = Overall_2, sc = Curr_21), code = plot_JS_code)
        callback_line_12 = CustomJS(args = dict(source = Overall_2, sc = Curr_22), code = plot_JS_code)
        menu1 = Select(
            options = list(data_1['Borough'].unique()),
            value = df_ratings[self.r_col1].idxmax(),
            title = 'Change borough to update green plots',
            background = '#46c284')
        menu2 = Select(
            options = list(data_1['Borough'].unique()), value = df_ratings[self.r_col1].idxmin(),
            title = 'Change borough to update red plots',
            background = '#ff5e76')

        p = figure(
            title = title_text,
            x_axis_label = None,
            y_axis_label = y_lbl,
            x_axis_type="datetime",
            x_range = (data_1['Date'].min(), self.x_pl1.max()))
        
        p.title.align = 'center'
        circle_1 = p.circle(x ='Date', y ='Value', color = good_col, source = Curr_11, alpha = 0.2, size = size_s)
        circle_2 = p.circle(x ='Date', y ='Value', color = bad_col, source = Curr_12, alpha = 0.2, size = size_s)
        line_11 = p.line(x ='Date', y ='Value', color = good_col, source = Curr_21, alpha = 1, line_width = size_l)
        line_12 = p.line(x ='Date', y ='Value', color = bad_col, source = Curr_22, alpha = 1, line_width = size_l)
        menu1.js_on_change('value', callback_circle_1)
        menu2.js_on_change('value', callback_line_11)
        menu1.js_on_change('value', callback_circle_2)
        menu2.js_on_change('value', callback_line_12)
        
        legend_title_1 = LegendItem(label = df_ratings[self.r_col1].idxmax())
        legend_circle_1 = LegendItem(label = 'historical values', renderers = [circle_1])
        legend_line_11 = LegendItem(label = 'trend', renderers=[line_11])
        legend_empty_line = LegendItem(label = ' ')
        legend_title_2 = LegendItem(label = df_ratings[self.r_col1].idxmin())
        legend_circle_2 = LegendItem(label = 'historical values', renderers = [circle_2])
        legend_line_12 = LegendItem(label = 'trend', renderers=[line_12])
        legend_JS_code = """
        var new_title = cb_obj.value;
        title.label = new_title;
        """
        callback_lt_1 = CustomJS(args = dict(title = legend_title_1), code = legend_JS_code)
        callback_lt_2 = CustomJS(args = dict(title = legend_title_2), code = legend_JS_code)
        menu1.js_on_change('value', callback_lt_1)
        menu2.js_on_change('value', callback_lt_2)
        legend_r_11 = LegendItem(label = 'Trend rating: {}'.format(df_ratings[self.r_col1].max()))
        legend_r_12 = LegendItem(label = 'Trend rating: {}'.format(df_ratings[self.r_col1].min()))
        lr_source_1 = ColumnDataSource(data = df_ratings.reset_index()[['Borough', self.r_col1]])
        lr_JS_code = """
        var borough = cb_obj.value;
        for(var i = 0; i <= source.get_length(); i++){
            if (source.data['Borough'][i] == borough){
                rating.label = string + source.data[column][i]
            }
        } 
        """
        callback_lr_11 = CustomJS(
            args = dict(source = lr_source_1, rating = legend_r_11, column = self.r_col1, string = 'Trend rating: '),
            code = lr_JS_code)
        callback_lr_12 = CustomJS(
            args = dict(source = lr_source_1, rating = legend_r_12, column = self.r_col1, string = 'Trend rating: ')
            , code = lr_JS_code)
        menu1.js_on_change('value', callback_lr_11)
        menu2.js_on_change('value', callback_lr_12)

        if plot2 != True:
            legend = Legend(
                items=[
                    legend_title_1,
                    legend_circle_1,
                    legend_line_11,
                    legend_r_11,
                    legend_empty_line,
                    legend_title_2,
                    legend_circle_2,
                    legend_line_12,
                    legend_r_12],
                label_width = 130,
                location = 'center_left',
                border_line_width = 0)        
        elif plot2 == True:
            data_3 = pd.DataFrame(
                self.y_pl2,
                index = df_boroughs.index,
                columns = self.x_pl2).stack().reset_index()
            data_3.columns = [data_3.columns[0], 'Date', 'Value']
            data_3c1 = data_3[data_3['Borough'] == df_ratings[self.r_col1].idxmax()]
            data_3c2 = data_3[data_3['Borough'] == df_ratings[self.r_col1].idxmin()]
            Overall_3 = ColumnDataSource(data = data_3)
            Curr_31 = ColumnDataSource(data = data_3c1)
            Curr_32 = ColumnDataSource(data = data_3c2)
            callback_line_21 = CustomJS(args = dict(source = Overall_3, sc = Curr_31), code = plot_JS_code)
            callback_line_22 = CustomJS(args = dict(source = Overall_3, sc = Curr_32), code = plot_JS_code)
            line_21 = p.line(
                x ='Date',
                y ='Value',
                color = good_col,
                source = Curr_31,
                alpha = 0.6, 
                line_width = size_l,
                line_dash = 'dashed')
            line_22 = p.line(
                x ='Date',
                y ='Value', 
                color = bad_col,
                source = Curr_32,
                alpha = 0.6,
                line_width = size_l,
                line_dash = 'dashed')
            menu1.js_on_change('value', callback_line_21)
            menu2.js_on_change('value', callback_line_22)
            legend_line_21 = LegendItem(label = 'long term trend', renderers=[line_21])
            legend_line_22 = LegendItem(label = 'long term trend', renderers=[line_22])
            
            legend_r_21_default = df_ratings.loc[df_ratings[self.r_col1].idxmax()][self.r_col2]
            legend_r_22_default = df_ratings.loc[df_ratings[self.r_col1].idxmin()][self.r_col2]
            legend_r_21 = LegendItem(label = 'Long term trend rating: {}'.format(legend_r_21_default))
            legend_r_22 = LegendItem(label = 'Long term trend rating: {}'.format(legend_r_22_default))
            lr_source_2 = ColumnDataSource(data = df_ratings.reset_index()[['Borough', self.r_col2]])
            legend = Legend(
                items=[
                    legend_title_1,
                    legend_circle_1,
                    legend_line_11,
                    legend_line_21,
                    legend_r_11,
                    legend_r_21,
                    legend_empty_line,
                    legend_empty_line,
                    legend_title_2,
                    legend_circle_2,
                    legend_line_12,
                    legend_line_22,
                    legend_r_12,
                    legend_r_22],
                label_width = 130,
                location = 'center_left',
                border_line_width = 0)
            callback_lr_21 = CustomJS(
                args = dict(
                    source = lr_source_2, 
                    rating = legend_r_21, 
                    column = self.r_col2, 
                    string = 'Long term trend rating: '),
                code = lr_JS_code)
            callback_lr_22 = CustomJS(
                args = dict(
                    source = lr_source_2,
                    rating = legend_r_22,
                    column = self.r_col2,
                    string = 'Long term trend rating: '),
                code = lr_JS_code)
            menu1.js_on_change('value', callback_lr_21)
            menu2.js_on_change('value', callback_lr_22)
        p.add_layout(legend, 'right')
        layout_row = b_row(menu1, menu2, align = 'center')
        layout = b_column(p, layout_row)
        show(layout)

# Function for analyzing k-mean clustering with various input parameters
def kmean_analysis_func(
    df, # dataframe to process
    n_cluster_min, # min of number of kmean clusters range to analyze
    n_cluster_max, # max of number of kmean clusters range to analyze
    feature_weights_list = 1): # weights list that will be applied to corresponding dataframe column

    # This loop evaluate kmeans for all the combinations of clusters number and random states
    # and stores best results according to silhouette score and relative plotting data
    df_weighted = df * feature_weights_list
    n_random_state = 20
    km_slh_score = np.zeros(
        (n_random_state,n_cluster_max + 1 - n_cluster_min)
        , dtype = float)
    km_score = np.zeros(
        (n_random_state,n_cluster_max + 1 - n_cluster_min)
        , dtype = float)
    max_s_score = 0
    for random_state in range (n_random_state):
        for n_cluster in range(n_cluster_min, n_cluster_max+1):
            km = KMeans(n_clusters = n_cluster, random_state = random_state).fit(df_weighted)
            km_score[random_state, n_cluster - n_cluster_min] = km.inertia_
            s_score = silhouette_score(df_weighted, km.labels_)
            km_slh_score[random_state, n_cluster - n_cluster_min] = s_score
            if s_score > max_s_score:
                max_s_score = s_score
                max_s_score_rs = random_state
                max_s_score_nc = n_cluster
    
    # Plot Silhouette and K-Mean Scores for all the number and a brief report
    fig, ax =plt.subplots(2, 1, figsize = size_p, dpi = dpi_p, sharex = 'col')
    fig.subplots_adjust(hspace = 0)
    ax[0].plot(range(n_cluster_min,n_cluster_max+1), km_slh_score[max_s_score_rs], color = good_col)
    ax[0].grid(b = True, which = 'both', axis = 'both', alpha = 0.3)
    ax[0].set_xlabel('Number of clusters', size = size_an)
    ax[0].set_xticks(range(n_cluster_min, n_cluster_max + 1))
    ax[0].set_ylabel('Silhouette Score',  size = size_an)
    ax[1].plot(range(n_cluster_min,n_cluster_max+1), km_score[max_s_score_rs], color = good_col)
    ax[1].grid(b = True, which = 'both', axis = 'both', alpha = 0.3)
    ax[1].set_xlabel('Number of clusters', size = size_an)
    ax[1].set_xticks(range(n_cluster_min, n_cluster_max + 1))
    ax[1].set_ylabel('Inertia', size = size_an)
    fig.suptitle('Kmean clustering analisys', size = size_t)
    plt.show()

    # print some analysis informations
    print('')
    print('Analisys report')
    print('Combinations of N° of clusters and N° of random states evaluated: {}'.format(
        n_random_state * (n_cluster_max - n_cluster_min + 1)))
    print('Best silhouette score: {}'.format(round(max_s_score,3)))
    print('Best silhouette score N° of clusters: {}'.format(max_s_score_nc))
    print('Best silhouette score random state: {}'.format(max_s_score_rs))
    print('')


# Function for further analyzing optimal k-means clustering
def kmean_model_func(
    df, # dataframe to process
    n_clusters, # number of optimal kmean cluster
    random_state, # optimal random state
    plot_rows, # plot grid number of rows
    plot_cols, # plot grid number of columns
    feature_weights_list = 1): # weights list that will be applied to corresponding dataframe column
    
    # Fits kmean model on weighted features
    df_weighted = df * feature_weights_list
    model = KMeans(n_clusters = n_clusters, random_state = random_state).fit(df_weighted)
    
    # Plot a grid of boxplots to evaluate clusters characteristics
    fig, ax = plt.subplots(
        plot_rows,
        plot_cols, 
        figsize = (size_p[0], 3/4 * size_p[1] * plot_rows),
        dpi = dpi_p,
        tight_layout = True, 
        squeeze = False)
    cluster_count = 0
    for row in range(plot_rows):
        for col in range(plot_cols):
            if cluster_count >= n_clusters:
                fig.delaxes(ax[row, col])
            else:
                ax[row, col].boxplot(df.loc[model.labels_ == cluster_count], vert = False)
                for spine in ax[row, col].spines.values():
                    spine.set_edgecolor('lightgray')
                ax[row, col].set_yticklabels(df.columns, size =  size_an - 1)
                ax[row, col].set_xlim(0,1)
                ax[row, col].set_xticks(np.arange(0,1.01,0.25))
                ax[row, col].set_title('Cluster {}'.format(cluster_count), size = size_an)
                ax[row, col].grid(b = True, which = 'both', axis = 'x', color = 'lightgray', linewidth = 1)
                ax[row, col].annotate(
                    'Cluster population: {}'.format(np.count_nonzero(model.labels_ == cluster_count)),
                    size = size_an-3,
                    xy = (0,-0.15),
                    xycoords = 'axes fraction')
                if col > 0:
                    ax[row, col].tick_params(
                        axis = 'y',
                        which = 'both',
                        labelleft = False)
                cluster_count += 1
    fig.suptitle('Ratings distribution in K-means clusters', y = 1.0, size = size_t)
    plt.show()
    return model


# Function for analyzing dbscan clustering with various input parameters
def dbscan_analysis_func(
    df, # dataframe to process
    eps_min, # min of dbscan distance range to process
    eps_max, # max of dbscan distance range to process
    eps_step, # step of dbscan distance range to process
    min_samples_list, # minimum number of samples to form a cluster
    color_list, # color list to apply to plots
    feature_weights_list = 1): # weights list that will be applied to corresponding dataframe column
    
    # This loop evaluate dbscan for all the combinations of eps and min_samples and stores
    # number of clusters formed and noise
    dbscan_noise_list = []
    dbscan_n_cluster_list = []
    dbscan_largest_cluster_size_list = []

    df_weighted = df * feature_weights_list
    for i in range(len(min_samples_list)):
        noise = []
        n_cluster = []
        largest_cluster_size = []
        for eps in np.arange(eps_min, eps_max, eps_step):
            model = DBSCAN(eps = eps, min_samples = min_samples_list[i]).fit(df_weighted)
            noise.append(np.count_nonzero(model.labels_ == -1))
            n_cluster.append(len(np.setdiff1d(model.labels_, -1)))
            non_noisy_labels = model.labels_[model.labels_ != -1]
            try:
                largest_cluster_size.append(np.unique(non_noisy_labels, return_counts = True)[1].max())
            except:
                largest_cluster_size.append(0)
        dbscan_noise_list.append(noise)
        dbscan_n_cluster_list.append(n_cluster)
        dbscan_largest_cluster_size_list.append(largest_cluster_size)

    # Plot number of clusters and noise for various eps and min_samples
    fig, ax =plt.subplots(3, 1, figsize = size_p, dpi = dpi_p, sharex = 'col')
    fig.subplots_adjust(hspace=0)
    for i in range(len(min_samples_list)):
        ax[0].plot(
            np.arange(eps_min, eps_max, eps_step),
            dbscan_n_cluster_list[i], 
            color = color_list[i])
        ax[1].plot(
            np.arange(eps_min, eps_max, eps_step),
            dbscan_noise_list[i],
            color = color_list[i],
            label = 'min samples = {}'.format(min_samples_list[i])) 
        ax[2].plot(
            np.arange(eps_min, eps_max, eps_step),
            dbscan_largest_cluster_size_list[i],
            color = color_list[i])
    
    for i in range(3):
        ax[i].grid(b = True, which = 'both', axis = 'both', alpha = 0.3)
        ax[i].set_xlabel('Epsilon', size = size_an)
        ax[i].set_xlim(eps_min, eps_max)
        ax[i].minorticks_on()

    
    ax[0].set_ylim(0)
    ax[0].set_ylabel('Clusters formed',  size = size_an)
    ax[1].yaxis.set_label_position("right")
    ax[1].yaxis.tick_right()
    ax[1].set_ylabel('Noise', size = size_an)
    ax[1].legend(fontsize = size_an-4, loc = 1)
    #ax[1].minorticks_on()
    ax[2].set_ylabel('Largest cluster size', size = size_an)
    #ax[2].minorticks_on()
    fig.suptitle('Dbscan clustering analysis', size = size_t)
    plt.show()
    
    # print some analysis informations
    print('')
    print('Analisys report')
    print('Combinations of N° of epsilon and N° of minimum samples evaluated: {}'.format(
        len(np.arange(eps_min, eps_max, eps_step)) * len(min_samples_list)))
    print('')

# Function for further analyzing optimal dbscan clustering
def dbscan_model_func(
    df, # dataframe to process
    eps, # chosen epsilon distance
    min_samples, # chosen minum number of samples to form a cluster
    plot_rows, # plot grid number of rows
    plot_cols, # plot grid number of columns
    feature_weights_list = 1): # weights list that will be applied to corresponding dataframe column

    # Fits dbscan model on weighted features
    df_weighted = df * feature_weights_list
    model = DBSCAN(eps = eps, min_samples = min_samples).fit(df_weighted)
    
    # Plot a grid of boxplots to evaluate clusters characteristics
    fig, ax = plt.subplots(
        plot_rows,
        plot_cols, 
        figsize = (size_p[0], 3/4 * size_p[1] * plot_rows), 
        dpi = dpi_p, 
        tight_layout = True,
        squeeze = False)
    cluster_count = 0
    for row in range(plot_rows):
        for col in range(plot_cols):
            if cluster_count >= np.count_nonzero(np.unique(model.labels_) != -1):
                fig.delaxes(ax[row, col])
            else:
                ax[row, col].boxplot(df.loc[model.labels_ == cluster_count], vert = False)
                for spine in ax[row, col].spines.values():
                    spine.set_edgecolor('lightgray')
                ax[row, col].set_yticklabels(df.columns, size =  size_an -1)
                ax[row, col].set_xlim(0,1)
                ax[row, col].set_xticks(np.arange(0,1.01,0.25))
                ax[row, col].set_title('Cluster {}'.format(cluster_count), size = size_an)
                ax[row, col].grid(b = True, which = 'both', axis = 'x', color = 'lightgray', linewidth = 1)
                ax[row, col].annotate(
                    'Cluster population: {}'.format(np.count_nonzero(model.labels_ == cluster_count)),
                    size = size_an-3,
                    xy = (0,-0.15),
                    xycoords = 'axes fraction')
                if col > 0:
                    ax[row, col].tick_params(
                        axis = 'y',
                        which = 'both',
                        labelleft = False)
                cluster_count += 1
    fig.suptitle('Ratings distribution in Dbscan clusters', y = 1.0, size = size_t)
    plt.show()
    # print some analysis informations
    print('')
    print('Analisys report')
    print('Total number of clustered samples: {}'.format(np.count_nonzero(model.labels_ != -1)))
    print('Number of unclustered samples: {}'.format(np.count_nonzero(model.labels_ == -1)))
    print('')

    return model

### First phase - Boroughs ratings

#### Collecting boroughs data
We start with the construction of a dataframe containing basic information about the 33 London boroughs:
- Borough Name (as index)
- Authority Code
- Area in square kilometres
- Population density esteem at current year

We will use the authority codes as a base to uniform boroughs naming among the datasets and the others for the valuation of the various grades.<br>
From this dataframe we derive also an empty dataframe we will fill with boroughs ratings in the various next steps.<br>
Below, the reader finds the aforementioned boroughs table.

In [ ]:
# BOROUGH AND RATINGS DATAFRAME

# We use basic boroughs information that are contained in the dataset we will use in the demographics analysis later
url_p = 'https://data.london.gov.uk/download/land-area-and-population-density-ward-and-borough/'
url = url_p + '77e9257d-ad9d-47aa-aeed-59a00741f301/housing-density-borough.csv' 
df_boroughs = pd.read_csv(url)
df_boroughs = df_boroughs.loc[
    (df_boroughs['Code'] >= 'E09000001') &
    (df_boroughs['Code'] <= 'E09000033') & 
    (df_boroughs['Year'] == c_date.year)][
    ['Code','Name','Square_Kilometres','Population_per_square_kilometre']]
df_boroughs.rename(columns={
    'Name': 'Borough',
    'Square_Kilometres': 'Km2',
    'Population_per_square_kilometre': 'Pop/Km2'},inplace = True)
df_boroughs.set_index('Borough', drop = True, verify_integrity = True, inplace = True)
df_boroughs.sort_index(inplace = True)
df_ratings = df_boroughs.drop(['Code', 'Km2', 'Pop/Km2'], axis = 1) # The ratings dataframe
df_boroughs

#### Housing market trends
##### *'Likely, if it went well in the past it will go well also in the immediate future'*

Analyzing the housing market historical series is surely important to comprehend the possible future trend.<br>
The importance of this dataset is in the fact that all the other data we will collect about boroughs have only a partial impact on the future housing market trends. In other words, there are surely other influential components that we cannot assess simply because they are not recorded and so they are not measurable. However, reading the historical housing market trends will give us information about how those hidden components acted in the past. Consequently, when we predict future based on these data, somehow we will keep them into account.

For this task we will use data from 1995 to 2020 available on London Datastore website.The main source of this dataset is the UK HM Land Registry. From this dataset we will acquire, for each borough, the historical average house sales prices and we will extrapolate future values using a polynomial regression model. Specifically we will fit two regression models for each borough using short term data and long term data. That will give us two regression curves that represent the short and the long term housing market trend. We will than acquire both curve point at future time to define a predicted value and finally we will calculate the relative increase from the current value to future value.

Finally we will assign two ratings scaling all the relative increases foreseen by the short term model so that the most performing borough will be graded one and the least performing will be graded zero. Of course we will do the same for the long term model. 

This phase is concluded with the recording of the so evaluated ratings in the ratings dataframe. 

In the following plots, we can see the various observations during the years and the second degree regression curves that best fits the data. Visually, the more the right end of the curves points up the better is the rating. The plots are interactive and two different boroughs can be selected to compare performances, by default the best and the worst performers by ratings are presented.

In [ ]:
# HOUSING MARKET DATAFRAME

url_p = 'https://data.london.gov.uk/download/uk-house-price-index/'
url = url_p + '70ac0766-8902-4eb5-aab5-01951aaed773/UK_House_price_index.xlsx'
df_hpi = pd.read_excel(url, sheet_name = 'Average price', usecols = 'A:AH', index_col = 0, skiprows = 1).transpose()
df_hpi.sort_index(inplace = True)
df_hpi.index = df_boroughs.index # We fix boroughs names in the index so that they match for all dataframes
df_hpi = round(df_hpi / 1000000, 3) # Transform average house prices in millions of GBP

In [ ]:
# HOUSING MARKET POLYNOMIAL REGRESSIONS AND RATINGS

housing_class = time_series_class(
    df = df_hpi,
    r_col1 = 'Housing short-term',
    r_col2 = 'Housing long-term')
housing_class.poly(
    pg_st = 2,
    num_years_st = 8, # Years of historical data we will fit predictive models on to catch SHORT-TERM trends
    pg_lt = 2,
    num_years_lt = 16) # Years of historical data we will fit predictive models on to catch LONG-TERM trends
housing_class.ts_plot(
    title_text = 'House prices, historical data and trends',
    y_lbl = 'Average house price in millions of GBP',
    plot2 = True)

#### Demographics
##### *'The more the buyers, the higher the prices'*
A population increase is a direct cause of residential properties demand increase and, on final, of housing market value increase.
This said, is important for our scope to verify demographics data.
We will do this using datasets from London Datastore which iclude, for each London borough, Greater London Authority demographics estimates (2016-based projections), 2011 Census and mid-year estimates by UK Office for National Statistics.
Luckly for us, this dataset contains also predictions on future years till 2050 so we will not need to use predictive models in this phase.
To grade each borough, we will simply evaluate the relative increase of population density, from the current date to the  future date, and we will perform the same scaling done in the previous phase. Finally we will store the data in the ratings dataframe.

In the following plots, we can see the various observations and predictions during the years and the slope of the line connecting the last two observations represents the relative increment. Visually, the more the line right end points up the better is the rating.
The plots are interactive and two different boroughs can be selected to compare performances, by default the best and the worst performers by ratings are presented.

In [ ]:
# DEMOGRAPHICS DATAFRAME

url_p = 'https://data.london.gov.uk/download/land-area-and-population-density-ward-and-borough/'
url = url_p + '77e9257d-ad9d-47aa-aeed-59a00741f301/housing-density-borough.csv'
df_demo = pd.read_csv(url)
df_demo = df_demo.loc[(df_demo['Code'] >= 'E09000001') &
                      (df_demo['Code'] <= 'E09000033') &
                      (df_demo['Year'] <= c_date.year+num_years_pr),
                      ['Name','Year','Population_per_square_kilometre']]
df_demo.rename(columns={'Name': 'Borough', 'Population_per_square_kilometre': 'Pop/Km2'}, inplace = True)
df_demo = df_demo.pivot(index = 'Borough', columns = 'Year', values = 'Pop/Km2')
df_demo.columns = pd.to_datetime(df_demo.columns, format = '%Y') # Convert columns labels to Timestamp for plotting

In [ ]:
# DEMOGRAPHICS INCREASES AND RATINGS

demo_class = time_series_class(
    df = df_demo,
    r_col1 = 'Demographic')
demo_class.rate()
demo_class.ts_plot(
    title_text = 'Population density, historical data and predictions',
    y_lbl = 'Population density per square kilometer',
    plot2 = False)

#### Income and earnings
##### *'The more the money, the higher the prices'*

In this paragraph we analyze trends of personal earnings from workers in different London boroughs. We assume that people tend to desire to live as near as possible to the workplace, so we will analyze a dataset that includes median income by wokplace and not by residence.

The dataset is provided by UK Office fo National Statistics (ONS) and provides information about earnings of employees who are working in an area, who are on adult rates and whose pay for the survey pay-period was not affected by absence.

As done previously, we will evaluate past trends with polynomial regression model and predict eventual short term future increases which we will then translate into boroughs ratings.

In the following plots, we can see the various observations during the years and the second degree regression curve that best fits the data. Visually, the more the right end of the curve points up the better is the rating. The plots are interactive and two different boroughs can be selected to compare performances, by default the best and the worst performers by ratings are presented.

In [ ]:
# INCOME DATAFRAME

url_p = 'https://data.london.gov.uk/download/earnings-workplace-borough/'
url = url_p + 'd3bfabd0-33cd-496c-8ab8-6edeb2227dc0/earnings-workplace-borough.xls'
df_income = pd.read_excel(url, sheet_name = 'FT workers annual Median', usecols = 'B:W', nrows = 34, na_values = '#')
df_income.drop(0, inplace = True)
df_income.set_index('Area', drop = True, inplace = True, verify_integrity = True)
df_income.index.name = 'Borough'
df_income.sort_index(inplace = True)
df_income.columns = pd.to_datetime(df_income.columns, format = '%Y')
df_income.interpolate(axis = 1, inplace = True) #We fill some nulls with a linear interpolation
df_income = round(df_income / 1000, 1) #Convert in thousands of GBP

In [ ]:
# INCOME POLYNOMIAL REGRESSION AND RATING

income_class = time_series_class(
    df = df_income,
    r_col1 = 'Area income')
income_class.poly(
    pg_st = 2,
    num_years_st = 21) # Years of historical data we will fit predictive models on to catch trend
income_class.ts_plot(
    title_text = 'Income, historical data and trends',
    y_lbl = 'Yearly median income in thousands GBP',
    plot2 = False)

#### Future residential development
##### *'The more the sellers, the lower the prices'*

An increase in housing availability in a specific area it's a negative factor because it will expand the offer and consequently will shrink the housing market value in that area. Therefore we need to know how many residential properties will be available in the future and, luckily for us, through the London Datastore we have access to the housing approvals recorded on the London Development Database (LDD).

The LDD contains details of all planning consents meeting criteria agreed with the London boroughs, who are responsible for submitting data to the database. Only planning consents are recorded on the database. For details of applications being considered by local planning authority (borough), or for refusals, we should visit each relevant planning authority’s website. For the sake of simplicity we will assume that the refusal rate is a constant percentage among all boroughs, thus will not affect our computations.

To rate each borough in this category we will assess how many permissions have been completed in the last 24 months. This data, considering building phase timings, will gives us a foresee about how many new residential units will be on the market in the next future. We also have to take into account that a given number of new residential properties impacts differently on each borough based on its demographics. Thus we will divide the above value by the borough population to obtain a parameter we can use to compare boroughs among themselves.

The bar plot below shows for each borough the number of new residential units for a thousand of inhabitants that are about to enter the housing market in the next years. The rating itself is also noted above each bar and represented by its color. Visually, the higher the bar, the worse the rating.

In [ ]:
# LAND DEVELOPMENT DATAFRAME

url_p = 'https://data.london.gov.uk/download/planning-permissions-on-the-london-development-database--ldd-/'
url = url_p + '4d65e64c-f8d8-4ca8-90cf-97921830cec2/LDD%20-%20Housing%20Approvals%20unit%20level.xlsx'

df_ldd = pd.read_excel(url, sheet_name = 'Approvals Unit Level', usecols = 'A,I,AF')
df_ldd.dropna(inplace = True)
df_ldd = df_ldd.loc[df_ldd['Completed date'] > (c_date - relativedelta(months = 24))]
df_ldd = df_ldd[['Borough','Net units']]
df_ldd = df_ldd.groupby('Borough').sum()
df_ldd.sort_index(inplace = True)
df_ldd['NewUnits/1000Pop'] = df_ldd['Net units'] / (df_boroughs['Km2'] * df_boroughs['Pop/Km2']) * 1000
df_ldd.drop('Net units', axis = 1, inplace = True)

In [ ]:
# LAND DEVELOPMENT RATING AND PLOTTING

df_ratings['Land development'] = MinMaxScaler().fit_transform(np.array(df_ldd))
df_ratings['Land development'] = round(df_ratings['Land development'], 2)

# This is a negative feature, so we have to invert the ratings (best = 0 and worst = 1)
df_ratings['Land development'] = - df_ratings['Land development'] + 1

# Bar Plot
plt.figure(figsize = size_p, dpi = dpi_p)
plt.title(
    'Future residential unit',
    y = 1.01,
    size = size_t)
plt.grid(b=True, which='both', axis='y', alpha = 0.5)

for i in range(33):
    plt.annotate(
        round(df_ratings.sort_values(['Land development'], ascending = True)['Land development'][i],1),
        xy = (i, df_ldd.sort_values(['NewUnits/1000Pop'], ascending = False)['NewUnits/1000Pop'][i] + 0.1),
        size = size_an-5,
        weight = 700,
        horizontalalignment = 'center',
        color = 'black')
plt.xlim(-0.5,32.5)
plt.xticks(rotation=90, size =  size_an)
plt.ylim(0,df_ldd['NewUnits/1000Pop'].max()+0.5)
plt.yticks(range(df_ldd['NewUnits/1000Pop'].max().astype(int)+1),size = size_an)

ax = sns.barplot(
    x = df_ldd.sort_values(['NewUnits/1000Pop'], ascending = False).index,
    y = 'NewUnits/1000Pop',
    data =  df_ldd.sort_values(['NewUnits/1000Pop'], ascending = False),
    palette = colors_func(df_ldd.sort_values(['NewUnits/1000Pop'], ascending = True)['NewUnits/1000Pop']))
ax.set_ylabel('New units per 1.000 inhabitants', size = size_an)
ax.set_xlabel('')

plt.show()

#### Crimes
##### *'The more the criminality, the lower the prices'*

It is glaring that crime level and land value are inversely proportional. To acquire crimes data we will use historical series provided by London Metropolitan Police Service through the London Datastore website. We will merge two time series, one for the last 24 months and the second for the less recent observations starting from April 2010.<br>
The aformantioned dataset will not contain information about the City of London borough becouse City of London Police is responsible for the safety of everyone in the 'Square Mile', not the Metropolitan Police Service.<br>
Since we want to complete the dataset, we will acquire this information from the first table in the webpage www.ukcrimestats.com/Police_Force/City_of_London_Police and merge the data.
The final dataset contains crime observations from December 2010 upwards in every London borough.

In the following plots, we can see the various observations during the years and the second degree regression curve that best fits the data. Visually, the more the right end of the curve points down the better is the rating.
The plots are interactive and two different boroughs can be selected to compare performances, by default the best and the worst performers by ratings are presented.

In [ ]:
# CRIME DATAFRAME

# We will have to merge 3 dataframes: 
# first one contains last 24 months data for Inner and Outer London boroughs
# second one contains before 31/12/2018 data for Inner and Outer London boroughs
# third one contains City of London borough data

# First dataframe
url_p = 'https://data.london.gov.uk/download/recorded_crime_summary/'
url = url_p + 'b03b8f4a-075f-4666-9c1d-8d9b0bfe3e63/MPS_Ward_Level_Crime_Historic_NewWard.csv'
df_crime_temp = pd.read_csv(url)
df_crime_temp.drop(['WardCode', 'Ward Name', 'Major Category', 'Minor Category'], axis = 1, inplace = True)
df_crime = df_crime_temp.groupby('Borough').sum()
df_crime.columns = pd.to_datetime(df_crime.columns, format = '%Y%m') # Convert columns labels to Timestamp

# Second dataframe
url = url_p + 'd2e9ccfc-a054-41e3-89fb-53c2bc3ed87a/MPS%20Borough%20Level%20Crime%20%28most%20recent%2024%20months%29.csv'
df_crime_temp = pd.read_csv(url)
df_crime_temp.drop(['MajorText', 'MinorText'], axis = 1, inplace = True)
df_crime_temp.rename(columns = {'LookUp_BoroughName':'Borough'}, inplace = True)
df_crime_temp = df_crime_temp.groupby('Borough').sum()
df_crime_temp.columns = pd.to_datetime(df_crime_temp.columns, format = '%Y%m') # Convert columns labels to Timestamp

# First two datasource columns, which are the survey dates, may overlap
columns_to_drop = df_crime_temp.columns[df_crime_temp.columns <= df_crime.columns.max()] # Create a list of overlapping columns
df_crime_temp.drop(columns_to_drop, axis = 1, inplace = True) # Drop the above list
df_crime = df_crime.merge(df_crime_temp, left_index = True, right_index = True)

# Third dataframe
url = 'https://www.ukcrimestats.com/Police_Force/City_of_London_Police'
df_crime_temp = pd.read_html(url)[1]
df_crime_temp.set_index('Unnamed: 0', drop = True, verify_integrity = True, inplace = True)
df_crime_temp = df_crime_temp.transpose()
df_crime_temp = df_crime_temp.loc['Total',:]
df_crime_temp.index = pd.to_datetime(df_crime_temp.index, format = '%b %Y') # Convert columns labels to Timestamp for plotting
df_crime_temp.name = 'City of London'
df_crime = df_crime.append(df_crime_temp, verify_integrity = True, sort = True)
df_crime.dropna(axis = 1, inplace = True) # We trim dataframe so that does non contains Nulls caused by different survey timings
df_crime.sort_index(inplace = True)

In [ ]:
# CRIME POLYNOMIAL REGRESSION AND RATING

crime_class = time_series_class(
    df = df_crime,
    r_col1 = 'Crime')
crime_class.poly(
    pg_st = 2,
    num_years_st = 10, # Years of historical data we will fit predictive models on to catch SHORT-TERM trends
    negative_feature = True)
crime_class.ts_plot(
    title_text = 'Crime records historical data and trend',
    y_lbl = 'Monthly total crimes',
    plot2 = False)

#### Air pollution
##### *'The more the smog, the lower the prices'*

We will use datasets from a study commissioned to the London King’s College London by Transport for London and the Greater London Authority.
You can read the full report here: https://www.london.gov.uk/what-we-do/environment/pollution-and-air-quality/modelling-long-term-health-impacts-air-pollution-london.<br>
This study used a computer simulation to estimate the long-term health impacts from 2016 to 2050 of the Ultra Low Emission Zone (ULEZ) and the wider suite of policies included in the London Environment Strategy (LES). Specifically, this study estimates the health impacts of the change in concentration of two pollutants: Nitrogen Dioxide (NO2) and Particulate Matter (PM2.5). These pollutants are known to have long-term health effects.<br>
For each borough we will pick the sum of both NO2 and PM2.5 related diseases incidence for each year from 2016 to 2021 and the rating will be evaluated considering the relative increse of incidence from 2020 value to 2021 value. Naturally the lower will be the increase, the better the rating.

In the following plots, we can see the various observations and predictions during the years and the slope of the line connecting the last two observations represents the relative increment. Visually, the more the line right end points down the better is the rating.
The plots are interactive and two different boroughs can be selected to compare performances, by default the best and the worst performers by ratings are presented.


In [ ]:
# AIR POLLUTION DATAFRAME

# We have to acquire data from 66 diffent urls, 2 for each borough (NO2 and PM25 predictions).
# To speed up processing time we have already stored all the 66 xlsm files.
# We will pick  NO2 and PM25 desease incidence for each borough, then we will sum both values
df_airpol = pd.DataFrame()
for borough in df_boroughs.index:
    df_temp1 = pd.read_excel('xls\{} NO2.xlsm'.format(borough), sheet_name = 'Incidence')
    df_temp1 = df_temp1.loc[
        (df_temp1['Scenario'] == 1) &
        (df_temp1['Year'] <= c_date.year+num_years_pr) &
        (df_temp1['Disease'] == 'total') &
        (df_temp1['AgeGroup'] == 'total')][
        ['Year','Incidence']]
    df_temp1.Year = pd.to_datetime(df_temp1.Year, format = '%Y')
    df_temp1.set_index('Year', drop = True, verify_integrity = True, inplace = True)
    df_temp2 = pd.read_excel('xls\{} PM25.xlsm'.format(borough), sheet_name = 'Incidence')
    df_temp2 = df_temp2.loc[
        (df_temp2['Scenario'] == 1) &
        (df_temp2['Year'] <= c_date.year+num_years_pr) &
        (df_temp2['Disease'] == 'total') &
        (df_temp2['AgeGroup'] == 'total')][
        ['Year','Incidence']]
    df_temp2.Year = pd.to_datetime(df_temp2.Year, format = '%Y')
    df_temp2.set_index('Year', drop = True, verify_integrity = True, inplace = True)
    df_temp = (df_temp1 + df_temp2).transpose()
    df_temp.rename(index={'Incidence': borough}, inplace = True)
    df_airpol = df_airpol.append(df_temp)
    df_airpol.index.name = 'Borough'

In [ ]:
# AIR POLLUTION INCREASES AND RATINGS

airpol_class = time_series_class(
    df = df_airpol,
    r_col1 = 'Air pollution')
airpol_class.rate(
    negative_feature = True)
airpol_class.ts_plot(
    title_text = 'Air pollution, historical data and predictions',
    y_lbl = 'Air pollution related diseases incidence',
    plot2 = False)

#### Boroughs ratings table

Now that all the data about boroughs have been collected and analyzed and boroughs performances have been measured, let's have a look at our final ratings dataframe displayed below.

In the next paragraph, we will try to use machine learning clustering algorithms to see if we can obtain some clusters populated with boroughs that are good candidates for further in-depth study. 

In [ ]:
# DISPLAY RATING DATAFRAME

df_ratings

#### Clustering boroughs

We will use K-means and DBSCAN algorithms for this task and, since they are both based on the concept of euclidean distance, we will use a weights system applied on the features that will shrink the variation range of those we will assume less important for our scenario. Indeed shrinking a feature range means that samples "diversity" (distance) in that particular feature will be reduced, so it will produce unlikely a new clusters. Samples that are different by reduced range features, will most likely end up spreading in clusters created by samples that are  different by full range features.

That said, according to our personal feelings about how the discussed topics could affect the housing market in the upcoming years, here is the weights we will use:

- Housing market short term trend: 100%
- Housing market long term trend: 75%
- Demographic trend: 100%
- Income trend: 50%
- New residential units: 50%
- Crime trend: 100%
- Air pollution trend: 25%

First let's have a look at K-means clustering. This algorithm aims to partition n observations into k clusters in which each observation belongs to the cluster with the nearest mean (cluster centroid) so as to minimize the within-cluster sum of squares (variance). Since the number of cluster has to be given as a starting parameter for the algorithm, a trial and error approach has to be taken to assess the optimal number of clusters to consider.

To do so, we will iterate the algorithm with various starting number of clusters and random centroids positions and we will measure its performances using two different metrics: Silhouette and Inertia.<br>
Silhouette score tells how far away the datapoints in one cluster are, from the datapoints in another cluster. The range of silhouette score is from negative one (worst) to to positive one (best). Values near zero indicate overlapping clusters. Negative values generally indicate that a sample has been assigned to the wrong cluster, as a different cluster is more similar.<br>
Inertia tells how far away the points within a cluster are. The range of inertia’s value starts from zero (best) and goes up.

In the picture below we can see the trends of inertia and silhouette score for various iterations of the K-means algorithm at different number of clusters values and random centroids starting positions. Only the random position that generate the best silhouette score is showed in the plot. Overall 140 iterations have been evaluated and we can identify two intersting points at number of clusters equal three and four. Both points have a silhouette score slightly above 0,3.
We think point at four is a bit more interesting even if it has a tiny smaller silhouette score. At this point the inertia is smaller and its plot shows also a small elbow. The elbow point in the inertia graph is interesting because after that the change in the value of inertia is less significant.

In [ ]:
# CLUSTERING ANALYSIS WITH KMEANS ALGORITHM #1

borough_weights = [1, 0.75, 1, 0.5, 0.5, 1, 0.25]

kmean_analysis_func(
    df = df_ratings,
    n_cluster_min = 2,
    n_cluster_max = 8,
    feature_weights_list = borough_weights)

In the following picture we further analyze the K-means algorithm output with number of clusters set at 4.
What we understand immediately, watching the ratings distribution in each cluster, is that there is not a group that performs well in all categories.<br>
Focusing on the more relevant features according to the previous weights defined, let's point out some clues we can see:
- Cluster N°0 contains boroughs with the best predictions about future criminality level. They also perform above average in the short-term housing market trend. It's likely that some interesting boroughs are in this cluster.
- Cluster N°1 is characterized by best performing boroughs in demographic trend and slightly above average criminality trend. Housing market trends are slightly sub average but we could find some interesting boroughs also in this cluster.
- Cluster N°2, the most populated, shows that majority of boroughs have above average housing market trends. Unfortunately they also show a sub average crime trend and demographic trend with some exceptions. Since is the most populated cluster, we can surely find some interesting boroughs in it.
- Cluster N°3 contains boroughs that perform sub average in every relevant features apart from long-term housing market trends that are in the average. It's unlikely that interesting boroughs are in this cluster.

In [ ]:
# CLUSTERING ANALYSIS WITH KMEANS ALGORITHM #2

kmean_model_boroughs = kmean_model_func(
    df = df_ratings,
    n_clusters = 4,
    random_state = 1,
    plot_rows = 2,
    plot_cols = 2,
    feature_weights_list = borough_weights)

Finally, let's have a look at the K-means clusters composition in the graph below.

In [ ]:
# CLUSTERING ANALYSIS WITH KMEANS ALGHORITHM #3

df = pd.DataFrame(data = kmean_model_boroughs.labels_, index = df_ratings.index, columns = ['Cluster'])
df.sort_values(['Cluster', 'Borough'], inplace = True)
df = pd.get_dummies(df, columns = ['Cluster'], prefix_sep = ' ').transpose()
df.replace({0: np.nan}, inplace = True)
plt.figure(figsize = size_p, dpi = dpi_p)
ax = plt.axes(frameon = True)
ax.set_title('K-means clusters composition', size = size_t)
ax.imshow(df, cmap = std_palette, aspect = 1.5, vmin = 0, vmax = 1)
ax.set_yticks(np.arange(df.shape[0]))
ax.set_yticks(np.arange(df.shape[0] + 1) -0.5, minor = True)
ax.set_yticklabels(df.index, size = size_an-1)
ax.set_xticks(np.arange(df.shape[1]))
ax.set_xticks(np.arange(df.shape[1] + 1) - 0.5, minor = True)
ax.set_xticklabels(df.columns, size = size_an-1)
ax.tick_params(axis = 'x', which = 'both', labelrotation = 90)
ax.tick_params(axis = 'both', which = 'minor', length = 0)


for i in range(df.shape[1]+1):
    ax.axvline(i-0.5, color = 'white', linewidth = 5)
        
for j in range(df.shape[0]+1):
    ax.axhline(j-0.5, color = 'white', linewidth = 5)
    
for i in range(df.shape[1]+1):
    ax.axvline(i-0.5, color = 'black', linewidth = 1)
        
for j in range(df.shape[0]+1):
    ax.axhline(j-0.5, color = 'black', linewidth = 1) 

plt.show()

Density-based spatial clustering of applications with noise (DBSCAN) is a density-based clustering non-parametric algorithm: given a set of points in some space, it groups together points that are closely packed together if their distance is less than "epsilon" and their number is more or equal to "min samples". It starts with an arbitrary starting point that has not been visited. This point's neighborhood is retrieved, and if it contains sufficiently many points, a cluster is started. Otherwise, the point is labeled as noise.

We will iterate through several values of epsilon and min samples values until we find a satisfying output. To monitor the quality of each iteration we will keep track of three parameters:

- Number of cluster formed
- Noise, that is the number of unclustered samples at the end of the process
- Largest cluster size

Increasing epsilon reduce the noise, because more and more isolated points are reached by larger and larger growing clusters, and augments the size of the largest cluster, since all the clusters formed tend to merge the one each other. If the data are effectively clusterable, we should find a trade-off value where the noise is low but the largest cluster size is not already too much high. 

The graph below shows the aforementioned parameters tendency at different values of epsilon and minimum sample size. We can obeserve how data tend to form a unique big cluster surrounded by noisy samples. We think an interesting point is at epsilon equal 0.41 and min samples equal to 2 with 3 clusters formed and noise equal to 6.

In [ ]:
# CLUSTERING ANALYSIS WITH DBSCAN ALGORITHM #1

dbscan_analysis_func(
    df = df_ratings,
    eps_min = 0.10,
    eps_max = 0.45,
    eps_step = 0.01,
    min_samples_list = [2,3,4,5],
    color_list = [good_col, 'blue', bad_col, 'orange'],
    feature_weights_list = borough_weights)

In the following picture we further analyze the DBSCAN algorithm output with the aforementioned parameters.

Focusing on the more relevant features we can see:
- Cluster N°0, the most populated, shows that majority of boroughs have above average housing market trends. Unfortunately they also show a sub average crime trend and demographic trend with some exceptions. We expect its composition to be similar to K-means cluster N°2.
- Cluster N°1, has two best performes in crime trend and average performers in housing market trend. Unfortunately these boroughs are also the worst performars in demographic trend. We expect its composition to be similar to K-means cluter N°0.
- Cluster N°2, has two boroughs also and they perform sub average in all relevant category except crime trend. It has no comparable K-means cluster.

In [ ]:
# CLUSTERING ANALYSIS WITH DBSCAN ALGORITHM #2

dbscan_model_boroughs = dbscan_model_func(
    df = df_ratings,
    eps = 0.41,
    min_samples = 2,
    plot_rows = 1,
    plot_cols = 3,
    feature_weights_list = borough_weights)

Again, let's have a look at the DBSCAN clusters composition in the graph below.

In [ ]:
# CLUSTERING ANALYSIS WITH KMEANS ALGHORITHM #3

df = pd.DataFrame(data = dbscan_model_boroughs.labels_, index = df_ratings.index, columns = ['Cluster'])
df.sort_values(['Cluster', 'Borough'], inplace = True)
df = pd.get_dummies(df, columns = ['Cluster'], prefix_sep = ' ').transpose()
df.index = ['Not clustered', df.index[1], df.index[2], df.index[3]]
df.replace({0: np.nan}, inplace = True)
df.loc['Not clustered'].replace({1: 0}, inplace = True)
plt.figure(figsize = size_p, dpi = dpi_p)
ax = plt.axes(frameon = True)
ax.set_title('DBSCAN clusters composition', size = size_t)
ax.imshow(df, cmap = std_palette, aspect = 1.5, vmin = 0, vmax = 1)
ax.set_yticks(np.arange(df.shape[0]))
ax.set_yticks(np.arange(df.shape[0] + 1) -0.5, minor = True)
ax.set_yticklabels(df.index, size = size_an-1)
ax.set_xticks(np.arange(df.shape[1]))
ax.set_xticks(np.arange(df.shape[1] + 1) - 0.5, minor = True)
ax.set_xticklabels(df.columns, size = size_an-1)
ax.tick_params(axis = 'x', which = 'both', labelrotation = 90)
ax.tick_params(axis = 'both', which = 'minor', length = 0)


for i in range(df.shape[1]+1):
    ax.axvline(i-0.5, color = 'white', linewidth = 5)
        
for j in range(df.shape[0]+1):
    ax.axhline(j-0.5, color = 'white', linewidth = 5)
    
for i in range(df.shape[1]+1):
    ax.axvline(i-0.5, color = 'black', linewidth = 1)
        
for j in range(df.shape[0]+1):
    ax.axhline(j-0.5, color = 'black', linewidth = 1) 

plt.show()

#### Borough "Residential Investment Score"

So far we have understood that each boroughs has some pros and cons and similarly the clusters formed by the two algorithm used. To pick some interesting boroughs to further investigate in the residential properties evaluation stage, we need to summarize the various ratings for each one of them so that we end up with an overall measure.<br>
We will call this measure the "Residential Investment Score" (RIS) and it will be the weighted sum of all the previous ratings, using the weights system already defined. The score itself will be scaled with a min-max normalization in the zero-one range, one will be the best and zero the worst.

With this new parameter it's now straightforward how to pick the right boroughs. Indeed in the bar plot below, we can see the RIS score calculated for every borough on the y axis. For visual reference, the highest and greenest the bar the better the score.

As we can see we have an uncontested winner by a margin of more than 20%.

Focusing on the best performers above average RIS score (0,6) we can see that they are included in all K-means clusters apart N°3, as predicted, and in DBSCAN clusters N°0 and N°1.

In [ ]:
# CALCULATE AND STORE RIQS

ris_list = list()
for borough in df_ratings.index:
    ris = 0
    for feature, weight in zip(df_ratings.columns, borough_weights):
        ris = ris + df_ratings[feature][borough] * weight
    ris_list.append(ris)
    
ris_list = MinMaxScaler().fit_transform(np.array(ris_list)[:, np.newaxis])

df_ris = pd.DataFrame(data = ris_list, index = df_ratings.index, columns = ['Residential Investment Score'])

plt.figure(figsize = size_p, dpi = dpi_p)
plt.title(
    'Residential Investment Score',
    y = 1.01,
    size = size_t)
plt.grid(b = True, which = 'minor', axis = 'y', alpha = 0.5)
plt.grid(b = True, which='major', axis='y', alpha = 1)
plt.xlim(-0.5, 32.5)
plt.xticks(rotation=90, size =  size_an)
plt.ylim(0, 1)
plt.yticks(np.arange(0,1.1,0.1), size = size_an)
plt.minorticks_on()
plt.tick_params(axis = 'x', which = 'minor', bottom = False)

ax = sns.barplot(
    x = df_ris.sort_values(by = 'Residential Investment Score', ascending = False).index,
    y = 'Residential Investment Score',
    data = df_ris.sort_values(by = 'Residential Investment Score', ascending = False),
    palette = colors_func(df_ris.sort_values(by = 'Residential Investment Score', ascending = False)['Residential Investment Score']))
ax.set_ylabel('')
ax.set_xlabel('')

plt.show()

#### Final boroughs ratings summary

Before going deep into the residential properties analysis phase, let's have an overlook at what we have finally found about boroughs features and their relations to future housing market value.

In the bar plots below we can see the ratings achieved by each borough, the ratings mean value and the Residential Investment Score (RIS). These plots are presented in an interactive form and the reader can choose which borough to display for a one to one detailed comparison. Initially, the best and the worst RIS performer are shown by default.

In [ ]:
# PLOTTING RATING SUMMARY

# Create Data Source for menu interaction with JavaScript directly in web browser
data_ratings = df_ratings.stack().reset_index()
data_ratings.columns = ['Borough', 'Feature', 'Rating']
data_ratings_best = data_ratings[data_ratings['Borough'] == df_ris['Residential Investment Score'].idxmax()]
data_ratings_worst = data_ratings[data_ratings['Borough'] == df_ris['Residential Investment Score'].idxmin()]
CDS_ratings = ColumnDataSource(data = data_ratings)
CDS_ratings_best = ColumnDataSource(data = data_ratings_best)
CDS_ratings_worst = ColumnDataSource(data = data_ratings_worst)
data_ris = df_ris.reset_index()
data_ris['x_coords'] = [0] * 33
data_ris = data_ris.append(data_ris).reset_index(drop = True)
data_ris.iloc[33:66]['x_coords'] = 7
data_ris_best = data_ris[data_ris['Borough'] == df_ris['Residential Investment Score'].idxmax()]
data_ris_worst = data_ris[data_ris['Borough'] == df_ris['Residential Investment Score'].idxmin()]
CDS_ris = ColumnDataSource(data = data_ris)
CDS_ris_best = ColumnDataSource(data = data_ris_best)
CDS_ris_worst = ColumnDataSource(data = data_ris_worst)

p1 = figure(
    plot_width = int(size_p[0] * dpi_p / 2),
    x_range = df_ratings.columns.to_list(),
    x_axis_label = None,
    x_minor_ticks = None,
    y_range = (0, 1.1),
    y_minor_ticks = None)
p1.xaxis.major_label_orientation = np.pi/2
p1.xgrid.visible = False
p2 = figure(
    plot_width = int(size_p[0] * dpi_p / 2),
    x_range = df_ratings.columns.to_list(),
    x_axis_label = None,
    x_minor_ticks = None,
    y_range = (0,1.1),
    y_minor_ticks = None,
    y_axis_location = 'right')
p2.xaxis.major_label_orientation = np.pi/2
p2.xgrid.visible = False

plot1 = p1.vbar(x = 'Feature', top = 'Rating', width = 0.3, color = good_col, source = CDS_ratings_best)
RIS_line1 = p1.line(x = 'x_coords',
                    y = 'Residential Investment Score',
                    color = good_col,
                    line_width = size_l,
                    line_dash = 'dashed',
                    source = CDS_ris_best)
p1.add_layout(RIS_line1)
legend1 = Legend(
    items=[LegendItem(label = 'Residential Investment Score', renderers = [RIS_line1])],
    location = (80,287),
    border_line_width = 0,
    background_fill_alpha = 0)
p1.add_layout(legend1)
subtitle_1 = Title(
    text = df_ris['Residential Investment Score'].idxmax(),
    align = 'center', 
    text_font_size = '{}pt'.format(size_an))
p1.add_layout(subtitle_1, 'above')

plot2 = p2.vbar(x = 'Feature', top = 'Rating', width = 0.3, color = bad_col, source = CDS_ratings_worst)
RIS_line2 = p2.line(x = 'x_coords',
                    y = 'Residential Investment Score',
                    color = bad_col,
                    line_width = size_l,
                    line_dash = 'dashed',
                    source = CDS_ris_worst)
p2.add_layout(RIS_line2)
legend2 = Legend(
    items=[LegendItem(label = 'Residential Investment Score', renderers = [RIS_line2])],
    location = (80,287),
    border_line_width = 0,
    background_fill_alpha = 0)
p2.add_layout(legend2)
subtitle_2 = Title(
    text = df_ris['Residential Investment Score'].idxmin(),
    align = 'center',
    text_font_size = '{}pt'.format(size_an))
p2.add_layout(subtitle_2, 'above')

# Code for JavaScript plot interactivity in web browser
plot_JS_code = """
var borough = cb_obj.value
sc.data['Feature']=[]
sc.data['Rating']=[]
for(var i = 0; i <= source.get_length(); i++){
    if (source.data['Borough'][i] == borough){
        sc.data['Feature'].push(source.data['Feature'][i])
        sc.data['Rating'].push(source.data['Rating'][i])
    }
}   
sc.change.emit();
"""
callback_1 = CustomJS(args = dict(source = CDS_ratings, sc = CDS_ratings_best),  code = plot_JS_code)
callback_2 = CustomJS(args = dict(source = CDS_ratings, sc = CDS_ratings_worst), code = plot_JS_code)

ris_line_JS_code = """
var borough = cb_obj.value
sc.data['x_coords']=[]
sc.data['Residential Investment Score']=[]
for(var i = 0; i <= source.get_length(); i++){
    if (source.data['Borough'][i] == borough){
        sc.data['x_coords'].push(source.data['x_coords'][i])
        sc.data['Residential Investment Score'].push(source.data['Residential Investment Score'][i])
    }
}   
sc.change.emit();
"""
callback_ris_line1 = CustomJS(args = dict(source = CDS_ris, sc = CDS_ris_best), code = ris_line_JS_code)
callback_ris_line2 = CustomJS(args = dict(source = CDS_ris, sc = CDS_ris_worst), code = ris_line_JS_code)

subtitle_JS_code = """
var new_title = cb_obj.value;
subtitle.text = new_title;
"""
callback_subtitle_1 = CustomJS(args = dict(subtitle = subtitle_1), code = subtitle_JS_code)
callback_subtitle_2 = CustomJS(args = dict(subtitle = subtitle_2), code = subtitle_JS_code)

menu3 = Select(
    options = list(data_ratings['Borough'].unique()),
    value = df_ris['Residential Investment Score'].idxmax(),
    title = 'Change borough to update green plots',
    background = '#46c284')
menu4 = Select(
    options = list(data_ratings['Borough'].unique()),
    value = df_ris['Residential Investment Score'].idxmin(),
    title = 'Change borough to update red plots',
    background = '#ff5e76')

menu3.js_on_change('value', callback_1)
menu4.js_on_change('value', callback_2)
menu3.js_on_change('value', callback_ris_line1)
menu4.js_on_change('value', callback_ris_line2)
menu3.js_on_change('value', callback_subtitle_1)
menu4.js_on_change('value', callback_subtitle_2)

title = Div(text = '<p style="font-size:{}px">Boroughs ratings summary</p>'.format(size_t + 2), align = 'center')
layout = b_row(b_column(p1, menu3), b_column(p2, menu4))
layout.spacing = 10
show(b_column(title, layout))

In the choropleth map below we can further visualize how each borough score in the various category that we can choose interactively by the legend radio buttons. Hovering the mouse on a borough, we can display all its ratings.

Inspecting this map we can discover how the geography influences some way the various performances.<br>
We can see:
- how the outer boroughs perform better than the central ones in the housing market trend features. 
- how the demographic best increase is predicted in few boroughs, namely "Hammersmith and Fulham" and "Ealing", with all the rest in the sub-average area.
- how a concentrated drop in income is predicted in "Redbridge" and "Barking and Dagenham", with all the rest in the over-average area.
- how residential land development (per inhabitants) is focused on the central boroughs of "Tower Hamlets" and "City of London".
- how crime trend improvement is better in few central boroughs while all the rest are in the sub-average area.
- how the air pollution trend in better in the south-eastern boroughs rather than in the north-western ones.

Finally, we can see how the RIS score tells us that, with some exceptions, outer boroughs seem more promising than inner boroughs for a residential investment.

In [ ]:
# BOROUGH RATINGS SUMMARY WITH FOLIUM MAP

geo_data = 'https://skgrange.github.io/www/data/london_boroughs.json'
geojson_data = json.loads(requests.get(geo_data).text)

df_ratings_riqs = df_ratings.join(df_ris, on = df_ratings.index)

# Let's add boroughs ratings to geojson data for tooltip information
for i in range(len(geojson_data['features'])):
    geojson_data['features'][i]['properties']['Borough'] =  geojson_data['features'][i]['properties']['name'] # create borugh label
    del(geojson_data['features'][i]['properties']['name']) # delete name label
    for j in df_ratings_riqs.columns:
        data_to_add = round(df_ratings_riqs.loc[geojson_data['features'][i]['properties']['Borough']][j], 2)
        geojson_data['features'][i]['properties'][j] = data_to_add

center_coord = [51.500, 0.05]

folium_figure = folium.Figure(
    width = size_p[0] * dpi_p,
    height = size_p[1] * dpi_p)

map = folium.Map(
    location = center_coord,
    tiles = None,
    control_scale = True,
    zoom_control = False,
    zoom_start = 10)

map.add_to(folium_figure)

folium.TileLayer('CartoDB positron', name = 'With map', control = False, opacity = 0.8).add_to(map)

first = True
for feature in df_ratings_riqs.columns:
    choropleth = folium.Choropleth(
        geo_data = geojson_data,
        name = feature,
        data = df_ratings_riqs,
        columns = [df_ratings_riqs.index, feature],
        key_on = 'feature.properties.Borough',
        fill_color = std_palette,
        fill_opacity = 0.5,
        line_opacity = 1,
        bins = 5,
        overlay = False,
        show = False,
        legend_name = 'Rating value')

    if first == False:
        for key in choropleth._children:
            if key.startswith('color_map'):
                del(choropleth._children[key])
        choropleth.add_to(map)
    else:
        choropleth.add_to(map)
    first = False

folium.GeoJson(
    geojson_data,
    tooltip = folium.GeoJsonTooltip(
        fields = [
            'Borough', 
            df_ratings_riqs.columns[0],
            df_ratings_riqs.columns[1],
            df_ratings_riqs.columns[2],
            df_ratings_riqs.columns[3],
            df_ratings_riqs.columns[4], 
            df_ratings_riqs.columns[5],
            df_ratings_riqs.columns[6],
            df_ratings_riqs.columns[7]]),
    style_function = lambda x: {
        'fillOpacity': 0,
        'weight': 0,
        'pane': 'tooltipPane'},
    control = False,
    overlay = True
    ).add_to(map)

folium.LayerControl(collapsed = False).add_to(map)

folium_figure

Now that we have an overall measure of boroughs performance in term of residential investment quality, we can pick those we will focus on in the next stage.<br>
We decided to consider only boroughs with a RIS score above 0,6. In the graph below we have a final summary view of those.

In [ ]:
# TOP BOROUGHS

riqs_score_min = 0.6
df_top_boroughs = df_ratings_riqs.loc[
    df_ratings_riqs['Residential Investment Score'] > riqs_score_min].sort_values(
    'Residential Investment Score', ascending = False)
plt.figure(figsize = size_p, dpi = dpi_p)
ax = plt.axes(frameon = False)
ax.set_title('Top boroughs with RIS over {}'.format(riqs_score_min), size = size_t)
ax.imshow(df_top_boroughs.transpose(), cmap = std_palette, alpha = 0.5)
ax.set_yticks(range(df_top_boroughs.shape[1]))
ax.set_yticklabels(df_ratings_riqs.columns.to_list(), size = size_an)
ax.set_xticks(range(df_top_boroughs.shape[0]))
ax.set_xticklabels(df_top_boroughs.transpose().columns, size = size_an)
ax.tick_params(axis = 'x', which = 'both', labelrotation = 90)
ax.tick_params(axis = 'both', which = 'both', length = 0)

for i in range(df_top_boroughs.transpose().shape[0]):
    for j in range(df_top_boroughs.transpose().shape[1]):
        ax.annotate(
            round(df_top_boroughs.transpose().iloc[i,j],1),
            xy = (j, i),
            size = size_t,
            horizontalalignment = 'center',
            verticalalignment = 'center',
            color = 'black',
            weight = 0)

for i in range(df_top_boroughs.transpose().shape[1]+1):
    ax.axvline(i-0.5, color = 'white', linewidth = 7)
    
for j in range(df_top_boroughs.transpose().shape[0]+1):
    ax.axhline(j-0.5, color = 'white', linewidth = 7)
    
for i in range(df_top_boroughs.transpose().shape[1]-1):
    ax.axvline(i+0.5, color = 'black', linewidth = 1)

ax.axhline(df_top_boroughs.transpose().shape[0]-1.5, color = 'black', linewidth = 1)


plt.show()

### Second phase - Residential properties selection and evaluation

#### Acquisition of on sales residential properties data

For this task we will use a web scraping algorithm applied on the Rightmove website. We will pick residential properties on sales in the London area with these characteristics:

- New home, recently built
- Selling price between 50.000 and 5.000.000 GBP
- Number of beds less or equal 5
- Land and park home property type excluded
- Buying schemes exclude

Data acquired are:

- Rightmove Id. number, for inspecting the property on Rightmove website using "https://www.rightmove.co.uk/properties/{ID_NUMBER}#/" without curly brackets.
- Address
- Type
- Number of bedrooms
- Size in square feet
- Price in millions of GBP
- Latitude
- Longitude

We first filter the data removing rows containing null values and duplicates. Then, size is converted to square meters and four new columns are computed:

- thousands GBP per square meter, which gives us information on the "affordability" of the property
- Square meters per bedroom, which gives us information on the "spaciousness" of the property
- Borough, to transfer on the residential properties all the information acquired in the previous phase
- Straight distance in kilometers from London "center", which we will assume are the location coordinates of Trafalgar Square. 

Finally, a check on size value is performed removing unlikely square meters per bedrooms values and properties outside interesting boroughs are removed.

Below the reader finds a brief report of the aforementioned process and the first five records of the dataframe.

In [ ]:
# CREATING PROPERTIES DATAFRAME FROM BS4 JSON FILES

# The web scraping algorithm used to create json files can retrieve a maximum of about
# a thousand properties per iteration subdivided in no more than 42 pages containig 25
# properties each and then saves each page as a json file. In this notebook we use the
# presaved json files. For interested readers, the algorithm itself is available in this
# project directory.

# Create a file_name list with all files found in json/houses directory
houses_file_list = []
for file_name in os.listdir('json/houses/'):
    houses_file_list.append(file_name)

print('')
print('Properties data acquisition report')
print('Number of json files (Rightmove search pages) read: {}'.format(len(houses_file_list)))

# Read and elaborate data in each file and progressively stores output in a dataframe        
columns_list = ['Id', 'Address', 'Type', 'Bedrooms', 'Size', 'Price', 'Latitude', 'Longitude']
df_properties = pd.DataFrame(columns = columns_list)
for file_name in houses_file_list:
    try: 
        file = open('json/houses/' + file_name, "r")
        jsonObj = json.loads(file.read())
    except:
        file = open('json/houses/' + file_name, "r", encoding="utf-8")
        jsonObj = json.loads(file.read())
    file.close()
    for item in jsonObj['properties']:
        size = ''.join(i for i in item['displaySize'] if i.isdigit())
        try:
            size = float(size)/10.7639
        except:
            size = np.nan
        data = {
            0: [item['id'],
            item['displayAddress'],
            item['propertySubType'],
            item['bedrooms'],
            size,
            item['price']['amount'] / 1000000,
            item['location']['latitude'],item['location']['longitude']]}
        df_row = pd.DataFrame.from_dict(data, orient = 'index', columns = columns_list)
        df_properties = df_properties.append(df_row)

print('Total number of properties acquired: {}'.format(df_properties.shape[0]))

# We eliminate duplicates
df_properties.drop_duplicates(subset = ['Id'], inplace = True, ignore_index = True)
df_properties.dropna(inplace = True)
print('Total number of properties without duplicates and "null" values: {}'.format(df_properties.shape[0]))


# We use property id as index
df_properties.set_index('Id', drop = True, verify_integrity = True, inplace = True)
df_properties.sort_index(inplace = True)

# We do some type conversion
df_properties['Bedrooms'] = df_properties['Bedrooms'].astype(int)
df_properties['Price'] = df_properties['Price'].round(3)
df_properties['Size'] = df_properties['Size'].round(1)

# We create two new columns that we will use to rank each house
df_properties['Size/bedroom'] = df_properties['Size'] / df_properties['Bedrooms']
df_properties['Size/bedroom'] = df_properties['Size/bedroom'].round(1)
df_properties['Price/sqmt'] = df_properties['Price'] * 1000 / df_properties['Size']
df_properties['Price/sqmt'] = df_properties['Price/sqmt'].round(2)

# We drop some rows with unlikely size values and not interesting types
df_properties.drop(
    df_properties.loc[
        (df_properties['Size/bedroom'] > 100) |
        (df_properties['Size/bedroom'] < 10)]
    .index, inplace = True)

df_properties.drop(
    df_properties.loc[
        (df_properties['Type'] == 'Plot') |
        (df_properties['Type'] == 'Off-Plan')]
    .index, inplace = True)
print('Total number of properties without unlikely size values and not interesting types: {}'.format(df_properties.shape[0]))

# We assign the proper borough for each house.
# Since we have the properties coordinates and we can obtain a london boroughs border map in geojson standard, we will
# use Shapely library to check geometrically which borough border polygon includes the location point.
top_boroughs_geojson = json.loads(
    requests.get('https://skgrange.github.io/www/data/london_boroughs.json').text)

# We delete geojson data of not interesting boroughs.
j = 0
for i in range(len(top_boroughs_geojson['features'])):
    if top_boroughs_geojson['features'][j]['properties']['name'] in df_top_boroughs.index:
        j += 1
    else:
        del top_boroughs_geojson['features'][j]

# We drop properties that are not in interesting boroughs
df_properties['Borough'] = None
df_properties['Distance'] = 0.0 # Distance from london center
london_center = (51.5048, -0.1235) # Coordinates of Trafalgar Square
for property in df_properties.iterrows():
    point = Point(property[1][6], property[1][5])
    for feature in top_boroughs_geojson['features']:
        polygon = shape(feature['geometry'])
        if polygon.contains(point):
            df_properties.at[property[0],'Borough'] = feature['properties']['name']            
            df_properties.at[property[0],'Distance'] = round(
                distance.distance(
                    london_center,
                    (df_properties.at[property[0],'Latitude'], df_properties.at[property[0],'Longitude'])).km, 1)
            break
df_properties.dropna(inplace = True)

print('Total number of properties inside interesting boroughs: {}'.format(df_properties.shape[0]))

print('')
df_properties.head()

As we can see from the brief report, we ended with a sample of 1376 "valid" properties on a total of 4243 sale adverts acquired from Rightmove.<br>
This reduction is due to the fact that many sale adverts do not provide size values in the proper field and this information should be acquired in other ways. For example, many adverts place the size data directly in the description of the properties, so we should define a text search algorithm to extract it. Considering the scope of this document, we will stay with the above sample and ignore properties with no size data recorded in the proper field. <br>
Of course not all the properties found are located in the aforementioned interesting boroughs, so the final count is 504 properties that are usable for the next steps.

In the histograms below we can observe distribution of boroughs and house types among the residential properties population. We can observe that:

- more than an half of the properties are located in the boroughs of Hackney and Lambeth
- vast majority of the sample contains flats and apartments

In [ ]:
# PROPERTIES STATISTICS 1

df = df_properties[[
    'Borough', 
    'Type',
    'Bedrooms',
    'Size', 
    'Price',
    'Size/bedroom',
    'Price/sqmt', 
    'Distance']]

fig, ax = plt.subplots(
        1,
        2, 
        figsize = (size_p[0], size_p[1]), 
        dpi = dpi_p, 
        tight_layout = True,
        squeeze = False)
fig.suptitle('Distribution of boroughs and types among properties', y = 1, size = size_t)
bins = 10

for i in range(2):
    ax[0, i].bar(
        df[df.columns[i]].value_counts().index,
        df[df.columns[i]].value_counts().values,
        width = 1,
        color = good_col,
        edgecolor = 'black',
        linewidth = 0.5,
        alpha = 0.5)
    
    ax[0, i].tick_params(axis = 'x', which = 'major', labelrotation = 90)
    ax[0, i].tick_params(axis = 'x', which = 'major', length = 0)

    ax[0, i].set_xlim(- 0.5, len(df.iloc[:,i].unique()) - 0.5)
    ax[0, i].set_ylim(0, df[df.columns[i]].value_counts().max())    
    ax[0, i].set_ylabel('Properties %')
    ax[0, i].set_xlabel('')
    ax[0, i].minorticks_on()
    ax[0, i].set_yticks(np.arange(0, df[df.columns[i]].value_counts().max(), df.shape[0] / 10))
    ax[0, i].set_xticks(np.arange(- 0.5, len(df.iloc[:,i].unique()) - 0.5, 1), minor = True)
    ax[0, i].yaxis.set_major_formatter(mtick.PercentFormatter(df.shape[0]))
    ax[0, i].set_axisbelow(True)
    if i > 0:
        ax[0, i].set_ylabel('')
    ax[0, i].grid(b = True, which = 'minor', axis = 'x', alpha = 0.3)
    ax[0, i].grid(b = True, which = 'major', axis = 'y', alpha = 1)
    ax[0, i].grid(b = True, which = 'minor', axis = 'y', alpha = 0.3)

plt.show()

We can see now distributions of other features. The width of each bar of the histograms below represents a range of values whose limits are indicated on the x axis. The ranges are intended starting value included and ending value excluded. The height of the darker green bar represents the percentage of properties with given value in that range and the height of the lighter green bar is the cumulative output. We can observe that:

- almost an half of the sample is represented by properties with two beds
- vast majority of the properties have size between 50 and 100 square meters
- over an half of the properties has a price between 500.000 and 1.000.000 GBP
- majority of the properties have size per bedroom in the range between 30 and 40 square meters
- almost 70% of the properties has a price per square meter in the range between 5.000 and 15.000 GBP
- over an half of the sample is within a distance of less than 6 kilometers from the London center

In [ ]:
# PROPERTIES STATISTICS 2

fig, ax = plt.subplots(
        3,
        2, 
        figsize = (size_p[0], size_p[1] * 2), 
        dpi = dpi_p, 
        tight_layout = True,
        squeeze = False)
fig.suptitle('Distribution of other features among properties', y = 1, size = size_t)
df_col = 2
bin_widths = (None, None, 1, 50, 0.5, 10, 5, 2)
for row in range(3):
    for col in range(2):
        sns.histplot(
            data = df,
            x = df.columns[df_col],
            label = df.columns[df_col],
            binwidth = bin_widths[df_col],
            binrange = (0, df.iloc[:,df_col].max()+bin_widths[df_col] / 2),
            alpha = 0.5,
            ax = ax[row, col],
            color = good_col,
            linewidth = 0.5)
        sns.histplot(
            data = df,
            x = df.columns[df_col],
            label = df.columns[df_col],
            binwidth = bin_widths[df_col],
            binrange = (0, df.iloc[:,df_col].max()+bin_widths[df_col] / 2),
            alpha = 0.2,
            ax = ax[row, col],
            color = good_col,
            cumulative = True,
            linewidth = 0)
        ax[row, col].set_xlim(0, df.iloc[:,df_col].max() + bin_widths[df_col] / (1 if bin_widths[df_col] == 1 else 2))
        ax[row, col].set_ylim(0, df.shape[0])
        ax[row, col].set_ylabel('Properties %')
        ax[row, col].set_xlabel('{}'.format(df.columns[df_col]))
        ax[row, col].minorticks_on()

        ax[row, col].set_xticks(
            np.arange(
                0,
                df.iloc[:,df_col].max() + bin_widths[df_col],
                bin_widths[df_col]))

        ax[row, col].set_xticks(
            np.arange(
                0,
                df.iloc[:,df_col].max() + bin_widths[df_col],
                bin_widths[df_col]),
            minor = True)

        ax[row, col].set_yticks(np.arange(0, df.shape[0]+1, df.shape[0] / 4))
        ax[row, col].set_yticks(np.arange(0, df.shape[0]+0.1, df.shape[0] / 20), minor = True)
        ax[row, col].set_axisbelow(True)
        ax[row, col].yaxis.set_major_formatter(mtick.PercentFormatter(df.shape[0]))
        if col > 0:
            ax[row, col].tick_params(axis = 'y', which = 'both', labelleft = False)
            ax[row, col].set_ylabel('')
        ax[row, col].grid(b = True, which = 'minor', axis = 'y', alpha = 0.3)
        ax[row, col].grid(b = True, which = 'major', axis = 'x', alpha = 0.3)
        ax[row, col].grid(b = True, which = 'major', axis = 'y', alpha = 1)
        ax[row, col].tick_params(axis = 'x', which = 'major', width = 0)
        df_col += 1

plt.show()

#### Acquisition of residential properties surroundings data

Now, we will acquire data about each property surroundings thanks to the Foursquare developer API.<br>
Foursquare is the most trusted, independent location data platform for understanding how people move through the real world. It combines the rich attributes of over 105 million global points-of-interest with the understanding of human movement from over 500 million devices.<br>
The developer API grant the user the ability to acquire, via coding, the surrounding venues given a location coordinates and a search radius. Also it consents to refine queries so that the output contains only venues of a given category.
We will feed the API with properties location coordinates, set the serch radious at 400 meters (5 minutes walking) and an output limit of 100 venues for each category and location. Finally, for every property, we will run a search iteration for each of these category:

- "Entertainment", corresponding to the merging of "Arts & Entertainment" and "Outdoors & Recreation" Foursquare categories
- "Service", corresponding to the merging of "College & University", "Professional & Other Places",  and "Shop & Service" Foursquare categories
- "Food & Night", corresponding to the merging of "Food" and "Nightlife Spot" Foursquare categories
- "Transport", corresponding to the "Travel & Transport" Foursquare categories

In [ ]:
# FOURSQUARE API DATA ACQUISITION

# API settings
CLIENT_ID = 'INSERT YOU CLIENT ID HERE'
CLIENT_SECRET = 'INSERT YOU CLIENT SECRET CODE HERE'
VERSION = '20180605'
limit = 100
radius = 400

# Combining default Foursquare API categories...
FS_categories = ['4d4b7104d754a06370d81259', # Arts & Entertainment -> Outdoors & Entertainment
              '4d4b7105d754a06372d81259', # College & University -> Services
              '4d4b7105d754a06374d81259', # Food -> Food and Nightlife                
              '4d4b7105d754a06376d81259', # Nightlife Spot -> Food and Nightlife
              '4d4b7105d754a06377d81259', # Outdoors & Recreation -> Outdoors & Entertainment             
              '4d4b7105d754a06375d81259', # Professional & Other Places -> Services
              '4d4b7105d754a06378d81259', # Shop & Service -> Services
              '4d4b7105d754a06379d81259'] # Travel & Transport -> Transports

# ...in our custom categories
venue_categories = ['{},{}'. format(FS_categories[0], FS_categories[4]),
                    '{},{},{}'. format(FS_categories[1], FS_categories[5], FS_categories[6]),
                    '{},{}'. format(FS_categories[2], FS_categories[3]),
                    FS_categories[7]]

# Set data acquisition algorithm behaviour:
# True = retrieve data from stored json files from previous API calls
# False = retrieve data from new API calls and update json files (in about 15 minutes...)
read_from_saved_files = True

df_venues = pd.DataFrame(index = df_properties.index, columns = ['Entertainment', 'Service', 'Food & Night', 'Transport'])
df_venues_details = pd.DataFrame()
for item in df_properties.iterrows():
    venue_count_list = []
    for j in range(len(venue_categories)):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&limit={}&radius={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            item[1]['Latitude'], 
            item[1]['Longitude'], 
            limit,
            radius,
            venue_categories[j])
        attempts = 0
        while attempts < 10:
            try:
                if read_from_saved_files == False:
                    result = requests.get(url).json()
                    json_file = open('json/venues/venues_Id{}_Cat{}.json'.format(item[0], j), 'w')
                    json.dump(result, json_file)
                    json_file.close()
                else:
                    json_file = open('json/venues/venues_Id{}_Cat{}.json'.format(item[0], j), 'r')
                    result = json.load(json_file)
                    json_file.close()
                venue_count_list.append(len(result['response']['groups'][0]['items']))
                break
            except:
                attempts += 1
        for i in range(len(result['response']['groups'][0]['items'])):
            df_venues_details = df_venues_details.append([[
                result['response']['groups'][0]['items'][i]['venue']['name'],
                df_venues.columns[j],
                result['response']['groups'][0]['items'][i]['venue']['location']['lat'],
                result['response']['groups'][0]['items'][i]['venue']['location']['lng']]])

    df_venues.loc[item[0]] = venue_count_list

df_venues = df_venues.astype(float)
df_venues_details.columns = ['Name', 'Category', 'Latitude', 'Longitude']
df_venues_details.drop_duplicates(subset = ['Name'], inplace = True, ignore_index = True)
df_venues_details.reset_index(drop = True, inplace = True)

In the histograms below we can observe the distribution of surrounding venues categories among the properties.
Each graph shows, in darker green, the number of properties in percentage at a given amount of surrounding venues. Every venues count was broken down in an appropriate number of bins, each one containing 5 counting unit. In lighter green, we can see the corresponding cumulative histograms.

We can observe:

- more than an half of properties has 5 or less entertainment venues surrounding and only 15% has more than 15
- above four fifth of the properties has 6 or more service venues, and over an half has 11 or more
- only slightly less than a fourth of the properties has 5 or less "Food & Night" venues while almost an half has more than 15
- Similarly, only a third of the properties has 5 or less "Transport" venues nearby

In [ ]:
# VENUES DISTRIBUTION

fig, ax = plt.subplots(
        2,
        2, 
        figsize = (size_p[0], size_p[1] * 2), 
        dpi = dpi_p, 
        tight_layout = True,
        squeeze = False)
fig.suptitle('Venues type distribution', y = 1, size = size_t)
binwidth = 5
df_venues_column = 0
for row in range(2):
    for col in range(2):
        sns.histplot(
            data = df_venues,
            x = df_venues.columns[df_venues_column],
            label = df_venues.columns[df_venues_column],
            binwidth = binwidth,
            alpha = 0.5,
            ax = ax[row, col],
            color = good_col,
            linewidth = 0.5)
        sns.histplot(
            data = df_venues,
            x = df_venues.columns[df_venues_column],
            label = df_venues.columns[df_venues_column],
            binwidth = binwidth,
            alpha = 0.2,
            ax = ax[row, col],
            color = good_col,
            cumulative = True,
            linewidth = 0)
        ax[row, col].set_xlim(0, df_venues.iloc[:,df_venues_column].max())
        ax[row, col].set_ylim(0, df_venues.shape[0])
        ax[row, col].set_ylabel('Properties %')
        ax[row, col].set_xlabel('{} venues count'.format(df_venues.columns[df_venues_column]))
        ax[row, col].minorticks_on()
        ax[row, col].set_xticks(np.arange(0, df_venues.iloc[:,df_venues_column].max() + binwidth, binwidth * 2))
        ax[row, col].set_xticks(np.arange(0, df_venues.iloc[:,df_venues_column].max() + binwidth, binwidth), minor = True)
        ax[row, col].set_yticks(np.arange(0, df_venues.shape[0]+1, df_venues.shape[0] / 4))
        ax[row, col].set_yticks(np.arange(0, df_venues.shape[0]+0.1, df_venues.shape[0] / 20), minor = True)
        ax[row, col].yaxis.set_major_formatter(mtick.PercentFormatter(df_venues.shape[0]))
        ax[row, col].set_axisbelow(True)
        if col > 0:
            ax[row, col].tick_params(
                axis = 'y',
                which = 'both',
                labelleft = False)
            ax[row, col].set_ylabel('')
        ax[row, col].grid(b = True, which = 'both', axis = 'both', alpha = 0.3)
        ax[row, col].grid(b = True, which = 'major', axis = 'y', alpha = 1)
        df_venues_column += 1


plt.show()

#### Residential properties and venues inspection with interctive map

In the map below we can see, with different markers colors, properties and surrounding venues collected in the previous stages.<br>
London boroughs are highlighted in light green and the interesting ones in darker green.<br>
Positions of properties are represented by black markers while venues are represented in the following colors: magenta for "Entertainment" venues, blue for "Service" venues, red for "Food & Night" venues and orange for "Transport" venues.

Map can be scaled and panned interactively and markers information can be obtained by hovering the mouse over them.

In [ ]:
# VISUALIZE PROPERTIES AND VENUES ON LONDON MAP

boroughs_geojson = json.loads(requests.get('https://skgrange.github.io/www/data/london_boroughs.json').text)

folium_figure = folium.Figure(
    width = size_p[0] * dpi_p,
    height = size_p[1] * dpi_p, title = 'TEST')

map = folium.Map(
    tiles = None,
    control_scale = True,
    zoom_control = False)

map.fit_bounds([
    [df_properties['Latitude'].min(), df_properties['Longitude'].min()],
    [df_properties['Latitude'].max(), df_properties['Longitude'].max()]],
    )

map.add_to(folium_figure)

folium.TileLayer('CartoDB positron', name = 'With map', control = False, opacity = 1).add_to(map)

folium.Choropleth(
    geo_data=boroughs_geojson,
    fill_opacity = 0.1,
    fill_color = good_col,
    line_opacity = 1,
    ).add_to(map)

folium.Choropleth(
    geo_data = top_boroughs_geojson,
    fill_opacity = 0.3,
    fill_color = good_col,
    line_opacity = 0,
    ).add_to(map)

folium.GeoJson(
    geojson_data,
    tooltip = folium.GeoJsonTooltip(
        fields = ['Borough']),
    style_function = lambda x: {
        'fillOpacity': 0,
        'weight': 0
        },
    control = False,
    overlay = True
    ).add_to(map)

for item in df_properties.iterrows():
    folium.CircleMarker(
        [item[1][5], item[1][6]],
        color = 'black',
        radius = 12,
        stroke = False,
        fill = True,
        fill_opacity = 0.6,
        tooltip = str(
            '<b>Property Id  </b> {}<br><b>Type  </b> {}<br><b>Bedrooms  </b> {}<br><b>Size  </b> {} sqmt<br><b>Price  </b> {} Mill.GBP<br>'.format(
                item[0],
                item[1][1],
                item[1][2],            
                item[1][3],
                item[1][4])),
        parse_html = False).add_to(map)
    
color_map = {'Entertainment': 'magenta',
             'Service': 'blue',
             'Food & Night': 'red',
             'Transport': 'orange'}
    
for item in df_venues_details.iterrows():
    folium.CircleMarker(
        [item[1][2], item[1][3]],
        color = color_map.get(item[1][1]),
        radius = 6,
        stroke = False,
        fill = True,
        fill_opacity = 0.6,
        tooltip = str(
            '<b>Venue Name  </b> {}<br><b>Type  </b> {}'.format(
                item[1][0],
                item[1][1])),
        parse_html = False).add_to(map)

folium_figure

#### Rating residential properties

To progress in our journey in the London residential housing market, we will create a properties "rating" dataframe similarly as we did for boroughs. For this task we will use these features:

- "Type", which will be a numerical conversion of the house types (the higher will be the value the more the prestige)
- "Spaciousness", which is the already defined Size/Bedroom (the higher the better)
- "Affordability", which is the opposite of Price/sqmt (the lower the price/sqmt the better the affordability)
- "Position", which is the opposite of the distance from London center (the less distance from center, the better)
- "Entertainments", "Services",	"Food & Night" and "Transports", which are the numebr of corresponding venues in five minutes walking range (the higher the better)
- "Borough RIS", the residential investment score of the belonging borough

All the features will be min-max normalized in the zero-one range and, below, the first five rows of this new dataframe.<br>
Also the RIS score, which was between 0.6 and 1.0 due the previous boroughs selection, has been normalized again to create more "distance" for the clustering algorithms we will use in the next stages of the analysis. As it has been clarified in the previous paragraphs, this is because we want to maximize the weight of this important feature.

In [ ]:
# DEFINING PROPERTIES RATINGS DATAFRAME

house_type_prestige = {'Apartment': 0.0,
                      'Flat': 0.0,
                      'Duplex': 1.0,
                      'Terraced': 1.0,
                      'Mews': 1.0,
                      'End of Terrace': 2.0,
                      'Town House': 2.0,
                      'Semi-Detached': 2.0,
                      'House': 3.0,
                      'Detached': 3.0,
                      'Penthouse': 3.0}

df_properties_ratings = pd.DataFrame(index = df_properties.index)
df_properties_ratings['Type'] = df_properties['Type'].replace(house_type_prestige)
df_properties_ratings['Spaciousness'] = df_properties['Size/bedroom']
df_properties_ratings['Affordability'] = - df_properties['Price/sqmt']
df_properties_ratings['Position'] = - df_properties['Distance']
df_properties_ratings[['Entertainments', 'Services', 'Food & Night', 'Transports']] = df_venues.iloc[:,0:4]
df_properties_ratings['Borough RIS'] = df_properties['Borough'].replace(df_ris.to_dict()['Residential Investment Score'])
df_properties_ratings = pd.DataFrame(
    data = MinMaxScaler().fit_transform(np.array(df_properties_ratings)),
    index = df_properties_ratings.index,
    columns = df_properties_ratings.columns)

df_properties_ratings.head(5)

#### Clustering residential properties

As did before with boroughs, we will use K-means and DBSCAN algorithms on the properties ratings dataframe to see if we can obtain some clusters populated with interesting properties and we will again apply some weights to the features before fitting the models. Ideally these weights represent a particular interest for a  feature we want to privilege. Apart from the fact that is objectively important to favor the RIS score since it synthesizes the probability of the selected properties value increase in the future, the weights of all the other features are subjective and depend on the investor profile.

Let's focus for example on an investor that want a very affordable solution in a good central position with sufficient services and some entertainment around. Of course, all while maximizing the quality of the investment. We could profile such an investor with the following weights:

- Type: 25%
- Spaciousness: 50%
- Affordability: 100%
- Position: 75%
- Entertainments: 25%
- Services: 50%
- Food & Night: 25%
- Transports: 50%
- Borough RIS: 100%

Going on with the clustering algorithms, as shown in the plots below, we find an optimum number of K-means clusters of four, with a relatively small elbow and a silhouette score of 0,334.

In [ ]:
# CLUSTERING ANALYSIS WITH KMEANS ALGORITHM #1

properties_weights = [0.25, 0.5, 1.0, 0.75, 0.25, 0.5, 0.25, 0.5, 1.0]

kmean_analysis_func(
    df = df_properties_ratings,
    n_cluster_min = 2,
    n_cluster_max = 8,
    feature_weights_list = properties_weights)

In the following picture we further analyze the K-means algorithm output with number of clusters set at 4.
Let's point out some clues we can see:

- Cluster N°0, is characterized by properties that are near the London center and have abundance of venues surrounding. As we may expect, they are also the least affordable properties even though there are some exceptions that could be be very interesting.
- Cluster N°1 contains more affordable properties that still are in a central position. This is the second best cluster regarding venues abundance.
- Cluster N°2 is somewhat similar to N°1. As we may see here the properties are less central and count less venues around but they are also more affordable.
- Cluster N°3 contains all the properties located in Havering. This is evident watching the Borough RIS parameter that is at its top in this borough. They are also among the most affordable but also the most isolated.

In [ ]:
# CLUSTERING ANALYSIS WITH KMEANS ALGORITHM #2

kmean_model_properties = kmean_model_func(
    df = df_properties_ratings,
    n_clusters = 4,
    random_state = 15,
    plot_rows = 2,
    plot_cols = 2,
    feature_weights_list = properties_weights)

Let's now try to form some clusters with the DBSCAN algorithm. As did before we will iterate through different values of epsilon and min samples and the graphs below shows the trends of number of clusters formed, noisy samples and largest cluster sizes.
We can see an interesting spot at epsilon equal 0.16 and min samples equal to 20. Indeed, looking the general trend of the plots, we can observe how a further increase of epsilon tends to rapidly create an unique large cluster while a decrease augment the noise level.

In [ ]:
# CLUSTERING ANALYSIS WITH DBSCAN ALGORITHM #1

dbscan_analysis_func(
    df = df_properties_ratings,
    eps_min = 0.08,
    eps_max = 0.25,
    eps_step = 0.005,
    min_samples_list = [6, 10, 14, 18, 20, 22],
    color_list = [good_col, 'blue', bad_col, 'orange', 'magenta', 'cyan'],
    feature_weights_list = properties_weights)

In the following picture we further analyze the DBSCAN algorithm output with the aforementioned values of epsilon and min samples.
Let's what clues we can find:

- Cluster N°0, is by far the most populated one. It's composition is characterized by properties with above average central position and affordability but lack of nearby venues.
- Cluster N°1 contains less affordable properties that are in a very central position and above average venues counts. Despite this, the affordability of these properties is in the average values so this cluster seems very interesting.
- Cluster N°2 groups the most affordable properties. We can see also less surrounding venues and a position in the average.
- Cluster N°3 is similar to N°2. The difference is that properties are located in the worst boroughs by RIS score (among those previously selected with RIS above 0.6). These properties are also more expensive than those grouped in cluster N°2 and in a better central position.
- Cluster N°4 is similar to N°3. We can observe a more central position and, coorespondingly a bigger presence of surrounding venues at the price of a slightly decrease of affordability.

In [ ]:
# CLUSTERING ANALYSIS WITH DBSCAN ALGORITHM #2

dbscan_model_properties = dbscan_model_func(
    df = df_properties_ratings,
    eps = 0.16,
    min_samples = 20,
    plot_rows = 2,
    plot_cols = 3,
    feature_weights_list = properties_weights)

## Results

After the clustering process we obtained some groups that seem to contain residential properties that match our investment preferences (read: weights). In particular K-means cluster N°0 and, more precisely, DBSCAN cluster N°1. Even if they are characterized by an average affordability level, which is our main concern according to the proposed weights, they are also notable for the outstanding central position of the properties contained and for the abundace of surrounding venues.

To define a final ranking we will proceed in the same way proposed for the definition of the RIS score. So, we will define a new properties feature that will synthetize all the others and will be obtained from the weighted sum of the other features. For this task we will use the original boroughs RIS scores and not the scaled ones used for the properties clustering processes.

In the table below we can see the final ranking of the first 25 properties ordered by descendind overall score and, as we may expect, they all belongs to the aforementioned clusters. 

In conclusion we can say that, for the kind of investor represented by the weights previously defined, the uncontested winner of The Battle of Neighborhoods is the borough of Lambeth, even if is not the most promising in term of investment quality. Indeed, still being a good choice from a pure real estate investment point of view, it prevails thanks to its outstanding central position and abundance of all kinds of venues at 5 mins walking range. Yet all this at a quite affordable price considering London center rates.

In [ ]:
# PROPERTY OVERALL SCORE AND FINAL TABLE

pos_list = list()
for residential_property in df_properties_ratings.index:
    pos = 0
    for feature, weight in zip(df_properties_ratings.columns, properties_weights):
        pos = pos + df_properties_ratings[feature][residential_property] * weight
    pos_list.append(pos)
    
pos_list = MinMaxScaler().fit_transform(np.array(pos_list)[:, np.newaxis])

df_results = pd.DataFrame(index = df_properties.index, data = df_properties[['Borough', 'Type']])
df_results['RIS'] = round(df_properties['Borough'].replace(df_ris.to_dict()['Residential Investment Score']),2)
df_results['Spaciousness'] = round(df_properties_ratings['Spaciousness'],2)
df_results['Affordability'] = round(df_properties_ratings['Affordability'],2)
df_results['Position'] = round(df_properties_ratings['Position'],2)
df_results['Entertainments'] = round(df_properties_ratings['Entertainments'],2)
df_results['Services'] = round(df_properties_ratings['Services'],2)
df_results['Food & Night'] = round(df_properties_ratings['Food & Night'],2)
df_results['Transports'] = round(df_properties_ratings['Transports'],2)
df_results['Overall Score'] = pos_list.round(2)
df_results['K-mean Cluster'] = kmean_model_properties.labels_
df_results['DBSCAN Cluster'] = dbscan_model_properties.labels_
df_results['DBSCAN Cluster'].replace(-1, 'unclustered', inplace = True)
df_results.sort_values('Overall Score', ascending = False, inplace = True)
df_results.head(25)

## Discussion

As we clearly understand, the final standing is heavily influenced by the weights proposed, first, to cluster boroughs and define RIS score, second, to cluster properties and define overall properties rating. So in this section we want to focus a bit more on these numbers.

The boroughs weights system is an objective product of "common sense" and it can be optimized. In other words we can say that there is a particular weights vector that, applied to the boroughs features, optimizes the ability of the model to predict the future trend of the housing market.<br>
Of course is not an easy task to find that particular vector and, somehow, the selection proposed in this document resemble a mere bet. A method to optimize these weights could be to reproduce all the analisys concerning the boroughs at a past date so that we can "measure" the final result with a known past value of the housing market trend.<br>
For example, we could choose as a current date December 2018, calculate all the boroughs ratings with predictions at one year, so December 2019, apply the weights system and compare the model perfomance with the real housing market value at December 2019. Iterating through this process for a sufficient amount of times could lead eventually to a certain weights vector that particularly optimizes the predictive performance of the model.

Of a totally different nature is the weights vector introduced to proceed through the residential properties selection. As stated previously, they synthetize the investor profile and his subjective investment preferences. In other words, changing this weights vector moves the point of view of the analysis.

Given that, we find particularly interesting to check the point of view of a pure investor whose goal is only to maximize the future returns.
For this task we could set a weights vector like below:

- Type: 0%
- Spaciousness: 10%
- Affordability: 100%
- Position: 10%
- Entertainments: 10%
- Services: 20%
- Food & Night: 10%
- Transports: 20%
- Borough RIS: 100%

Without going through the clustering processes, we will directly jump to calculate the Overall Score as did before. <br>
Looking at the results shown in the table below we can see how, from this different perspective, the boroughs of Havering, first, and Barnet, second, are more interesting.

In [ ]:
# ALTERNATIVE STANDINGS

properties_weights_2 = [0.0, 0.1, 1.0, 0.1, 0.1, 0.2, 0.1, 0.2, 1.0]

pos_list = list()
for residential_property in df_properties_ratings.index:
    pos = 0
    for feature, weight in zip(df_properties_ratings.columns, properties_weights_2):
        pos = pos + df_properties_ratings[feature][residential_property] * weight
    pos_list.append(pos)
    
pos_list = MinMaxScaler().fit_transform(np.array(pos_list)[:, np.newaxis])

df_results_2 = pd.DataFrame(index = df_properties.index, data = df_properties[['Borough', 'Type']])
df_results_2['RIS'] = round(df_properties['Borough'].replace(df_ris.to_dict()['Residential Investment Score']),2)
df_results_2['Spaciousness'] = round(df_properties_ratings['Spaciousness'],2)
df_results_2['Affordability'] = round(df_properties_ratings['Affordability'],2)
df_results_2['Position'] = round(df_properties_ratings['Position'],2)
df_results_2['Entertainments'] = round(df_properties_ratings['Entertainments'],2)
df_results_2['Services'] = round(df_properties_ratings['Services'],2)
df_results_2['Food & Night'] = round(df_properties_ratings['Food & Night'],2)
df_results_2['Transports'] = round(df_properties_ratings['Transports'],2)
df_results_2['Overall Score'] = pos_list.round(2)
df_results_2.sort_values('Overall Score', ascending = False, inplace = True)
df_results_2.head(25)

## Conclusion

Further analysis of the two previous tables lead to these final conclusions:

- From the perspective of a buyer that want to settle near London center without being subdued to its highest rates, the best solutions are in Lambeth borough
- From the perspective of a buyer that want exclusively to maximize the potential of his investment, the best solutions are in Havering borough
- The borough of Barnet represents a middleway option in almost every category. It is surely a first choice if properties in Lambeth are considered too much expensive.

We hope the reader enjoyed this journey in the London housing market and we reproduce below the links to the Rightmove website adverts corresponding to the aforementioned tables, for further information.

First table properties urls:
- https://www.rightmove.co.uk/properties/73540722#/
- https://www.rightmove.co.uk/properties/74253939#/
- https://www.rightmove.co.uk/properties/74253936#/
- https://www.rightmove.co.uk/properties/76403126#/
- https://www.rightmove.co.uk/properties/74253945#/
- https://www.rightmove.co.uk/properties/94206326#/
- https://www.rightmove.co.uk/properties/98334512#/
- https://www.rightmove.co.uk/properties/73897482#/
- https://www.rightmove.co.uk/properties/75437676#/
- https://www.rightmove.co.uk/properties/75277581#/
- https://www.rightmove.co.uk/properties/75306222#/
- https://www.rightmove.co.uk/properties/57137964#/
- https://www.rightmove.co.uk/properties/87440440#/
- https://www.rightmove.co.uk/properties/75306210#/
- https://www.rightmove.co.uk/properties/75277593#/
- https://www.rightmove.co.uk/properties/54826797#/
- https://www.rightmove.co.uk/properties/79398886#/
- https://www.rightmove.co.uk/properties/99835166#/
- https://www.rightmove.co.uk/properties/87105763#/
- https://www.rightmove.co.uk/properties/99778481#/
- https://www.rightmove.co.uk/properties/83142793#/
- https://www.rightmove.co.uk/properties/92009732#/
- https://www.rightmove.co.uk/properties/79398829#/
- https://www.rightmove.co.uk/properties/68802561#/
- https://www.rightmove.co.uk/properties/92010191#/

Second table properties urls:
- https://www.rightmove.co.uk/properties/101190266#/
- https://www.rightmove.co.uk/properties/97154141#/
- https://www.rightmove.co.uk/properties/97154852#/
- https://www.rightmove.co.uk/properties/97160357#/
- https://www.rightmove.co.uk/properties/97164023#/
- https://www.rightmove.co.uk/properties/100736492#/
- https://www.rightmove.co.uk/properties/99047096#/
- https://www.rightmove.co.uk/properties/94158866#/
- https://www.rightmove.co.uk/properties/62501793#/
- https://www.rightmove.co.uk/properties/94158869#/
- https://www.rightmove.co.uk/properties/94738283#/
- https://www.rightmove.co.uk/properties/64602930#/
- https://www.rightmove.co.uk/properties/73551661#/
- https://www.rightmove.co.uk/properties/99870122#/
- https://www.rightmove.co.uk/properties/98200358#/
- https://www.rightmove.co.uk/properties/67375443#/
- https://www.rightmove.co.uk/properties/65784120#/
- https://www.rightmove.co.uk/properties/99870134#/
- https://www.rightmove.co.uk/properties/99334322#/
- https://www.rightmove.co.uk/properties/67229934#/
- https://www.rightmove.co.uk/properties/94206326#/
- https://www.rightmove.co.uk/properties/62501271#/
- https://www.rightmove.co.uk/properties/67375446#/
- https://www.rightmove.co.uk/properties/98698949#/
- https://www.rightmove.co.uk/properties/74176698#/

In [ ]:
# PROPERTY URL LIST #1

print('')
print('First table properties urls:')
for Id in df_results.head(25).index:
    print('https://www.rightmove.co.uk/properties/{}#/'.format(Id))

In [ ]:
# PROPERTY URL LIST #2

print('')
print('Second table properties urls:')
for Id in df_results_2.head(25).index:
    print('https://www.rightmove.co.uk/properties/{}#/'.format(Id))